# Login

In [1]:
%matplotlib notebook
import os
import numpy as np
import matplotlib.pyplot as plt
import qulab
from qulab.tools.wavedata import *

In [2]:
#print(qulab.admin.get_database())
#qulab.admin.get_collection_info()
#qulab.admin.drop_collection(database='ten_qubits_measurement',
#                            collection=['application',])
qulab.admin.set_database('qubit_measurement')

The recent database is "qubit_measurement".
The following databases are already exist:
['admin', 'config', 'local', 'qubit_measurement', 'qubit_measurement_test']
Your recent selected database already is "qubit_measurement".


In [3]:
#qulab.admin.register()

In [4]:
qulab.login('liuqichun','123456')

# list all apps

In [ ]:
qulab.listApps()

# base app

## init config

In [119]:
import os
import datetime
import platform
import qulab
from pathlib import Path
from ruamel import yaml
from qulab.db.schema.app import listApplication

class meas_config(qulab.Application):
    async def work(self):
        setting_dict = {'config_dir': None,
                        'config_file': None,
                        'circuit_list': None,
                        'Load_config': True}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        if platform.system() in ['Darwin', 'Linux']:
            home = os.getenv('HOME')
        elif platform.system() == 'Windows':
            home = os.getenv('ProgramData')
        else:
            home = os.getcwd()
        
        if setting_dict['Load_config']:
            if setting_dict['config_dir'] and setting_dict['config_file']:
                config_path = os.path.join(config_dir,config_file)
                path = Path(config_path)
                config = yaml.safe_load(path.read_text())
            else:
                path = Path(os.path.join(home,'Quantum_Lab','meas_config_path.yaml'))
                meas_config_path = yaml.safe_load(path.read_text())
                config_path = os.path.join(meas_config_path['config_dir'],meas_config_path['config_file'])
                path = Path(config_path)
                config = yaml.safe_load(path.read_text())
        else:
            if (not setting_dict['config_dir']) or (not setting_dict['config_file']):
                config_info = None
                print('you must provide the config_dir and config_file for creating!')
            else:
                meas_config_path = os.path.join(home,'Quantum_Lab','meas_config_path.yaml')
                config = {'config_dir': setting_dict['config_dir'],
                          'config_file': setting_dict['config_file']}
                with open(meas_config_path, 'w', encoding='utf-8') as f:
                    yaml.dump(config, f, Dumper=yaml.RoundTripDumper)
                ##
                ##
                config_dir = setting_dict['config_dir']
                config_file = setting_dict['config_file']
                config_path = os.path.join(config_dir,config_file)
                circuit_list = setting_dict['circuit_list']
                config = dict()

                time_list = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
                config['update_time'] = time_list
                
                database_path = Path(os.path.join(home,'Quantum_Lab','config.yaml'))
                database = yaml.safe_load(database_path.read_text())
                config['database'] = database['db']['db']
                config['result_dir'] = os.path.join(config_dir,'measurement_result')
                #
                app_info = dict()
                app_list = list(listApplication())
                for app in app_list:
                    app_key = str(app).split(' ')[1].split('.')[1]+'_app'
                    app_value = str(app).split(' ')[1]
                    if 'auto_meas' not in app_value:
                        app_info[app_key] = app_value
                config['app_info'] = app_info
                #
                inst_info = {'dc': 'DC',
                             'awg': 'AWG',
                             'mw': 'MW',
                             'pna': 'PNA',
                             'ats': 'ATS',
                             'rsa': 'RSA'}
                config['inst_info'] = inst_info
                #
                jpa_info = {'bias_inst': 'DC',
                            'bias_ch': 1,
                            'bias_amp': 0.0,
                            'drive_mw': 'MW',
                            'drive_amp': 10,
                            'drive_freq': 12000000000.0}
                config['jpa_info'] = jpa_info
                #
                adc_info = {'n': 1024,
                            'ARange': 2.0,
                            'BRange': 2.0,
                            'repeats': 2048,
                            'maxlen': 2048,
                            'bufferCount': 2048,
                            'trigLevel': 0.5,
                            'triggerDelay': 0,
                            'triggerTimeout': 10,
                            'average': True}
                config['adc_info'] = adc_info
                #
                x6_info = {'n': 1024,
                           'repeats': 2048,
                           'sampleRate': 1e9,
                           'dac_outchannel': [1,1,1,1],
                           'dac_sRate': 0.5e9,
                           'triggerTimeout': 10,
                           'average': True}
                config['x6_info'] = x6_info
                #
                pna_info = {'Power': -40,
                            'center': 6500000000.0,
                            'span': 10.0,
                            'points': 3,
                            'Bandwidth': 100,
                            'Output': 'ON',
                            'digital_delay': 1e-08,
                            'phase_compensate': False,
                            'timeout': 200}
                config['pna_info'] = pna_info
                #
                bias_info = dict()
                mw_info = dict()
                mw_drive_info = dict()
                mw_meas_info = dict()
                awg_info = dict()
                awg_drive_info = dict()
                awg_drive_info['sRate'] = 1e9
                awg_drive_info['pulse_length'] = 100e-6
                awg_meas_info = dict()
                awg_meas_info['sRate'] = 1e9
                awg_meas_info['pulse_length'] = 100e-6
                awg_meas_info['meas_trig_source'] = 'AWG'
                awg_meas_info['mess_trig_ch']= 4
                awg_meas_info['meas_delay']= 0
                qubit_info = dict()
                for circuit in circuit_list:
                    bias_info[circuit] = {'inst_source': 'DC',
                                          'ch': 1,
                                          'amp': 0.0}
                    mw_drive_info[circuit] = {'inst_source': 'MW',
                                              'power': 0,
                                              'frequency': 4000000000.0,
                                              'carry': 50e6,
                                              'IQ_cali_array': [[1,0,0],[1,0,0]]}
                    mw_meas_info[circuit] = {'inst_source': 'MW',
                                             'power': 0,
                                             'frequency': 6000000000.0,
                                             'carry': 50e6,
                                             'IQ_cali_array': [[1,0,0],[1,0,0]]}
                    awg_drive_info[circuit] = {'inst_source': 'AWG',
                                               'I_ch': 1,
                                               'Q_ch': 2,
                                               'Z_ch': 3,
                                               'mk_ch':1,
                                               'Z_delay': 0,
                                               'mk_delay': 0}
                    awg_meas_info[circuit] = {'inst_source': 'AWG',
                                              'I_ch': 1,
                                              'Q_ch': 2,
                                              'trig_delay': 0}
                    qubit_info[circuit] = {'bare_freq': 6500000000.0,
                                           'disp_freq': 6500000000.0,
                                           'disp_amp': 0.2,
                                           'probe_freq': 6500000000.0,
                                           'probe_amp': 0.1,
                                           'readout_base': 1.0,
                                           'rabi_amp': 1.0,
                                           'f01': 4250000000.0,
                                           'an_harmonicity': 250000000.0,
                                           'pi_array': [2e-8,0.3,2e-8,0.15,2e-8,0.0],
                                           'DRAG_ratio': 0.0,
                                           'xy_pulse_type': 'CosPulse',
                                           'z_pulse_type': 'DC',
                                           'single_shot_array': [0,1,1,0]}
                config['bias_info'] = bias_info
                config['mw_info'] = {'drive_info': mw_drive_info,
                                     'meas_info': mw_meas_info}
                config['awg_info'] = {'drive_info': awg_drive_info,
                                      'meas_info': awg_meas_info}
                config['qubit_info'] = qubit_info
                #
                N = len(circuit_list)
                swap_gate = dict()
                cphase_gate = dict()
                couple_gate = dict()
                couple_control_info = dict()
                for idx in range(N-1):
                    temp = 'S'+str(idx)+'_'+str(idx+1)
                    swap_gate[temp] = [[3e-8,0.5],[3e-8,0],[3e-8,0]]
                    temp = 'CP'+str(idx)+'_'+str(idx+1)
                    cphase_gate[temp] = [[2e-8,0.5],[2e-8,0]]
                    temp = 'G'+str(idx)+'_'+str(idx+1)
                    couple_gate[temp] = [[2e-8,0.5],[2e-8,0]]
                    couple_control_info[temp] = {'inst_source': 'AWG1',
                                                 'ch': 1, 
                                                 'G_delay': 0}
                    #
                    temp = 'S'+str(idx+1)+'_'+str(idx)
                    swap_gate[temp] = [[3e-8,0.5],[3e-8,0],[3e-8,0]]
                    temp = 'CP'+str(idx+1)+'_'+str(idx)
                    cphase_gate[temp] = [[2e-8,0.5],[2e-8,0]]
                    temp = 'G'+str(idx+1)+'_'+str(idx)
                    couple_gate[temp] = [[2e-8,0.5],[2e-8,0]]
                    couple_control_info[temp] = {'inst_source': 'AWG1',
                                                 'ch': 1, 
                                                 'G_delay': 0}
                config['couple_control_info'] = couple_control_info
                config['two_bit_gate_info'] = {'swap_gate': swap_gate,
                                               'cphase_gate': cphase_gate,
                                               'couple_gate': couple_gate}
                #
                if os.path.exists(config_path):
                    config_dir = setting_dict['config_dir']
                    config_file = config_file.split('.')[0]+'_backup.yaml'
                    backup_config_path = os.path.join(config_dir,config_file)
                    if os.path.exists(backup_config_path):
                        os.remove(backup_config_path)
                    os.rename(config_path,backup_config_path)
                    print('The config file is already existed and has been renamed as "%s".' %config_file)
                    print('A new config file has been created.')
                else:
                    print('A new config file has been created.')
                with open(config_path, 'w', encoding='utf-8') as f:
                    yaml.dump(config, f, Dumper=yaml.RoundTripDumper)
        yield config

In [120]:
meas_config.save(package='base')

## drive sequence generate

In [8]:
import re
import qulab
import numpy as np
from qulab.tools.wavedata import *
from qulab.tools.cliffordgroup import *

class pulse_sequence_generate(qulab.Application):
    async def work(self):
        # 输入gate_list格式类似为 ['X0','X1','Y2p1',['X2n0','Y2'],'S0_1','CP1_2','G_0_1']
        # 其中内部的 [ ] 内门操作表示为同时进行的操作
        # 输出格式为:
        """
        pulse_sequence = {'Qubit1':{'XY':None,
                                    'Z': None,
                                    'XY_mk':None},
                          'Couple':{'G0_1':None,
                                    'G1_0':None}}
        """
        
        setting_dict = {'gate_list': None,
                        'qubit_info': None,
                        'two_bit_gate_info': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # qubit_info从外部输入，以便要扫描某些参数的app调用该app
        gate_list = setting_dict['gate_list']
        qubit_info = setting_dict['qubit_info']
        two_bit_gate_info = setting_dict['two_bit_gate_info']
        # 若为从外部输入qubit_info等，则从config_info内获取
        if not qubit_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            qubit_info = config['qubit_info']
            two_bit_gate_info = config['two_bit_gate_info']
        #默认采样率为1GS/s，当采样率为其它数值时，在将波形写入awg之前进行重采样即可
        sRate = 1e9
        #定义一个1ns的空脉冲用于将所有XY脉冲间隔开来
        B_0 = Blank(width=1e-9,sRate=sRate)
        
        # get pulse type
        if list(qubit_info.values())[0]['xy_pulse_type'] == 'DC':
            xy_type = DC
        else:
            xy_type = CosPulse
        if list(qubit_info.values())[0]['z_pulse_type'] == 'DC':
            z_type = DC
        else:
            z_type = CosPulse
        
        ## 根据gate_list信息初始化所有脉冲序列pulse_sequence
        pulse_sequence = dict()
        qubit_idx = list()
        couple_idx = list()
        gate_re = re.compile(r'^(I|S|CP|G|X2p|X2n|Y2p|Y2n|X|Y)(.*)')
        for gate in np.hstack(np.array(gate_list)):
            m = gate_re.search(gate)
            if 'S' in gate or 'CP' in gate:
                if ':' in m.group(2):
                    temp = m.group(2).split(':')[0]
                else:
                    temp = m.group(2)
                temp_idx = int(temp.split('_')[0])
                if temp_idx not in qubit_idx:
                    qubit_idx.append(temp_idx)
                temp_idx = int(temp.split('_')[1])
                if temp_idx not in qubit_idx:
                    qubit_idx.append(temp_idx)
            elif 'G' in gate:
                temp_idx = m.group(2)
                if temp_idx not in couple_idx:
                    couple_idx.append(temp_idx)
            else:
                temp_idx = int(m.group(2))
                if temp_idx not in qubit_idx:
                    qubit_idx.append(temp_idx)
        qubit_idx = np.sort(qubit_idx)
        for idx in qubit_idx:
            pulse_sequence['Qubit'+str(idx)] = {'XY': Wavedata(sRate=sRate),
                                                'Z': Wavedata(sRate=sRate),
                                                'XY_mk':Wavedata(sRate=sRate)}
        Couple = dict()
        for idx in couple_idx:
            Couple['G'+idx] = Wavedata(sRate=sRate)
        if len(couple_idx) > 0:
            pulse_sequence['Couple'] = Couple
        
        ## 定义处理单个门脉冲
        def process_single_gate(qubit_info = None,
                                two_bit_gate_info = None, 
                                gate = None,
                                pulse_sequence = None):
            m = gate_re.search(gate)
            if 'S' in gate or 'CP' in gate:
                if ':' in m.group(2):
                    temp = m.group(2).split(':')[0]
                    theta_idx = int(m.group(2).split(':')[1])-1
                else:
                    temp = m.group(2)
                    theta_idx = 0
                qubit_idx = int(temp.split('_')[0])
                if m.group(1) == 'S':
                    temp_pulse_length = two_bit_gate_info['swap_gate']['S'+temp][theta_idx][0]
                    temp_pulse = z_type(width=temp_pulse_length,sRate=sRate
                                       )*two_bit_gate_info['swap_gate']['S'+temp][theta_idx][1]
                else:
                    temp_pulse_length = two_bit_gate_info['cphase_gate']['CP'+temp][theta_idx][0]
                    temp_pulse = z_type(width=temp_pulse_length,sRate=sRate
                                       )*two_bit_gate_info['cphase_gate']['CP'+temp][theta_idx][1]
                pulse_sequence['Qubit'+str(qubit_idx)]['Z'] = pulse_sequence['Qubit'+str(qubit_idx)]['Z']|temp_pulse
            elif 'G' in gate:
                temp_pulse_length = two_bit_gate_info['couple_gate']['G'+m.group(2)][0]
                temp_pulse = z_type(width=temp_pulse_length,sRate=sRate
                                   )*two_bit_gate_info['couple_gate']['G'+m.group(2)][1]
                pulse_sequence['Couple']['G'+m.group(2)] = pulse_sequence['Couple']['G'+m.group(2)]|temp_pulse
            else:
                qubit_idx = int(m.group(2))
                if m.group(1) == 'I':
                    temp_pulse = Blank(width=qubit_info['Qubit'+str(qubit_idx)]['pi_array'][4],sRate=sRate
                                      )+qubit_info['Qubit'+str(qubit_idx)]['pi_array'][5]
                else:
                    temp_pulse = cliffordGroup.gen_XY(index = m.group(1),
                                                      pi_len = qubit_info['Qubit'+str(qubit_idx)]['pi_array'][0],
                                                      pi_factor = qubit_info['Qubit'+str(qubit_idx)]['pi_array'][1],
                                                      half_pi_len = qubit_info['Qubit'+str(qubit_idx)]['pi_array'][2], 
                                                      half_pi_factor = qubit_info['Qubit'+str(qubit_idx)]['pi_array'][3], 
                                                      sRate = sRate, 
                                                      TYPE = xy_type)
                    temp_pulse_drag = temp_pulse.derivative()
                    temp_pulse_drag = 1e-6*temp_pulse_drag
                    temp_pulse = temp_pulse+1j*temp_pulse_drag*qubit_info['Qubit'+str(qubit_idx)
                                                                         ]['DRAG_ratio']/float(qubit_info['Qubit'+str(qubit_idx)
                                                                                                         ]['an_harmonicity'])
                        
                temp_pulse = B_0|temp_pulse|B_0
                temp_pulse_length = len(temp_pulse.data)/sRate
                pulse_sequence['Qubit'+str(qubit_idx)]['XY'] = pulse_sequence['Qubit'+str(qubit_idx)]['XY']|temp_pulse
                pulse_sequence['Qubit'+str(qubit_idx)]['XY_mk'] = pulse_sequence['Qubit'+str(qubit_idx)
                                                                                ]['XY_mk']|DC(width=temp_pulse_length,sRate=sRate)
            return pulse_sequence
        
        ## 定义取所有脉冲序列长度的最大值
        def acquire_max_length(pulse_sequence = None):
            max_length = 0
            for circuit in pulse_sequence:
                if 'Qubit' in circuit:
                    temp_length = len(pulse_sequence[circuit]['XY'].data)
                    if max_length < temp_length:
                        max_length = temp_length
                    temp_length = len(pulse_sequence[circuit]['Z'].data)
                    if max_length < temp_length:
                        max_length = temp_length
                else:
                    for couple in pulse_sequence[circuit]:
                        temp_length = len(pulse_sequence[circuit][couple].data)
                        if max_length < temp_length:
                            max_length = temp_length
            return max_length
        
        ## 定义对小于最大值的脉冲序列进行补偿，使所有序列长度一致
        def ensure_same_length(qubit_info = None,
                               two_bit_gate_info = None, 
                               pulse_sequence = None, 
                               max_length = None):
            for circuit in pulse_sequence:
                if 'Qubit' in circuit:
                    temp_length = len(pulse_sequence[circuit]['XY'].data)
                    if max_length > temp_length:
                        pulse_sequence[circuit]['XY'] = pulse_sequence[circuit]['XY']|DC(width=(
                            max_length-temp_length)/sRate,sRate=sRate)*qubit_info[circuit]['pi_array'][-1]
                        pulse_sequence[circuit]['XY_mk'] = pulse_sequence[circuit]['XY_mk']|Blank(width=(
                            max_length-temp_length)/sRate,sRate=sRate)
                    temp_length = len(pulse_sequence[circuit]['Z'].data)
                    if max_length > temp_length:
                        pulse_sequence[circuit]['Z'] = pulse_sequence[circuit]['Z']|Blank(width=(
                            max_length-temp_length)/sRate,sRate=sRate)
                else:
                    for couple in pulse_sequence[circuit]:
                        temp_length = len(pulse_sequence[circuit][couple].data)
                        if max_length > temp_length:
                            pulse_sequence[circuit][couple] = pulse_sequence[circuit][couple]|DC(width=(
                                max_length-temp_length)/sRate,sRate=sRate)*two_bit_gate_info['couple_gate'][couple][-1]
            return pulse_sequence
        
        ## 处理所有脉冲操作
        ## 处理所有脉冲操作
        gate_re = re.compile(r'^(I|S|CP|G|X2p|X2n|Y2p|Y2n|X|Y)(.*)')
        for gate in gate_list:
            ## gate 为 list 结构（即['X','X','X']形式），表示gate里面所有的脉冲是同时进行的
            ## 当各脉冲长度不一致时，以最长的为准，将其它脉冲以默认幅度进行延长（通常默认幅度即为0）
            if isinstance(gate, list):
                for gate_child in gate:
                    # 处理单个脉冲
                    pulse_sequence = process_single_gate(qubit_info = qubit_info, 
                                                         two_bit_gate_info = two_bit_gate_info, 
                                                         gate = gate_child,
                                                         pulse_sequence = pulse_sequence)
                # 取所有脉冲序列长度的最大值
                max_length = acquire_max_length(pulse_sequence = pulse_sequence)
                # 对小于最大值的脉冲序列进行补偿，使所有序列长度一致
                pulse_sequence = ensure_same_length(qubit_info = qubit_info, 
                                                    two_bit_gate_info = two_bit_gate_info, 
                                                    pulse_sequence = pulse_sequence,
                                                    max_length = max_length)
            ## gate为单个脉冲操作
            else:
                # 处理单个脉冲
                pulse_sequence = process_single_gate(qubit_info = qubit_info, 
                                                     two_bit_gate_info = two_bit_gate_info, 
                                                     gate = gate,
                                                     pulse_sequence = pulse_sequence)
                # 取所有脉冲序列长度的最大值
                max_length = acquire_max_length(pulse_sequence = pulse_sequence)
                # 对小于最大值的脉冲序列进行补偿，使所有序列长度一致
                pulse_sequence = ensure_same_length(qubit_info = qubit_info, 
                                                    two_bit_gate_info = two_bit_gate_info, 
                                                    pulse_sequence = pulse_sequence,
                                                    max_length = max_length)
        yield pulse_sequence

In [9]:
pulse_sequence_generate.save(package='base')

## meas pulse generate

In [10]:
import qulab
import numpy as np
from qulab.tools.wavedata import *

class meas_pulse_generate(qulab.Application):
    async def work(self):
        setting_dict = {'circuit_list': None,
                        'meas_info': None,
                        'qubit_info': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        meas_info = setting_dict['meas_info']
        qubit_info = setting_dict['qubit_info']
        if not meas_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            meas_info = config['mw_info']['meas_info']
            qubit_info = config['qubit_info']
        sRate = 1e9
        # 默认测量脉冲为ADC采样长度加500个采样点，且至少 2us 长，
        if config['adc_info']['n'] < 2000:
            pulse_length = 2e-6
        else:
            pulse_length = (config['adc_info']['n']+500)/sRate
        
        meas_pulse = dict()
        for circuit in setting_dict['circuit_list']:
            temp_pulse = DC(width = pulse_length, sRate = sRate)
            meas_pulse[circuit] = vIQmixer.carry_wave(carry_freq = meas_info[circuit]['carry'], 
                                                      IQ = temp_pulse, 
                                                      carry_cali = meas_info[circuit]['IQ_cali_array'], 
                                                      DEG = True)*qubit_info[circuit]['probe_amp']
        yield meas_pulse

In [11]:
meas_pulse_generate.save(package='base')

## map drive sequence

In [121]:
import qulab
import numpy as np
from qulab.tools.wavedata import *

class map_drive_sequence(qulab.Application):
    async def work(self):
        setting_dict = {'drive_pulse': None,
                        'drive_info': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        drive_pulse = setting_dict['drive_pulse']
        if setting_dict['drive_info']:
            drive_info = setting_dict['drive_info']
        else:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            drive_info = config['awg_info']['drive_info']
        sRate = drive_info['sRate']
        pulse_length = drive_info['pulse_length']
        
        ## 若输入波形采样率和config里的采样率不一致，则按照config里的sRate重新采样
        ## 并对所有 XY 驱动进行载波调制
        for circuit in drive_pulse:
            if 'Qubit' in circuit:
                if drive_pulse[circuit]['XY'].sRate != sRate:
                    drive_pulse[circuit]['XY'] = drive_pulse[circuit]['XY'].resample(sRate=sRate)
                    drive_pulse[circuit]['Z'] = drive_pulse[circuit]['Z'].resample(sRate=sRate)
                    drive_pulse[circuit]['XY_mk'] = drive_pulse[circuit]['XY_mk'].resample(sRate=sRate)
                drive_pulse[circuit]['XY'] = vIQmixer.carry_wave(carry_freq = config['mw_info']['drive_info'][circuit]['carry'], 
                                                                 IQ = drive_pulse[circuit]['XY'], 
                                                                 carry_cali = config['mw_info']['drive_info'][circuit]['IQ_cali_array'],
                                                                 DEG = True)
            else:
                for couple in drive_pulse[circuit]:
                    if drive_pulse[circuit][couple].sRate != sRate:
                        drive_pulse[circuit][couple] = drive_pulse[circuit][couple].resample(sRate=sRate)
        ## 初始化一个waveform_collection dict，其中key包好awg source 和 channel信息
        ## value即为对应通道的波形
        waveform_collection = dict()
        waveform_collection_flag = dict()
        for circuit in drive_pulse:
            if 'Qubit' in circuit:
                # xy 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['I_ch'])+ '__' + str(drive_info[circuit]['Q_ch'])
                waveform_collection[wavename] = Wavedata(sRate=sRate)
                waveform_collection_flag[wavename] = 0
                # z 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['Z_ch'])
                waveform_collection[wavename] = Wavedata(sRate=sRate)
                waveform_collection_flag[wavename] = 0
                # xy_mk 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['mk_ch']) + '__mk'
                waveform_collection[wavename] = Wavedata(sRate=sRate)
                waveform_collection_flag[wavename] = 0
            else:
                # couple 控制脉冲
                for couple in drive_pulse[circuit]:
                    wavename = config['couple_control_info'][couple]['inst_source'] + '__' + str(config['couple_control_info'][couple]['ch'])
                    waveform_collection[wavename] = Wavedata(sRate=sRate)
                    waveform_collection_flag[wavename] = 0
        for circuit in drive_pulse:
            if 'Qubit' in circuit:
                ## xy 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['I_ch'])+ '__' + str(drive_info[circuit]['Q_ch'])
                waveform_collection_flag[wavename] = waveform_collection_flag[wavename]+1
                temp_pulse = drive_pulse[circuit]['XY']
                # 相同source和channel信号进行叠加，并且处理因叠加而造成的offset偏差
                waveform_collection[wavename] = waveform_collection[wavename]+temp_pulse
                if waveform_collection_flag[wavename] > 1:
                    I_offset = config['mw_info']['drive_info'][circuit]['IQ_cali_array'][0][1]
                    Q_offset = config['mw_info']['drive_info'][circuit]['IQ_cali_array'][1][1]
                    waveform_collection[wavename] = vIQmixer.carry_wave(carry_freq = 0, 
                                                                        IQ = waveform_collection[wavename], 
                                                                        carry_cali = [[1,-I_offset,0],[1,-Q_offset,0]],
                                                                        DEG = True)
                    waveform_collection_flag[wavename] = 1
                ## z 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['Z_ch'])
                waveform_collection_flag[wavename] = waveform_collection_flag[wavename]+1
                temp_pulse = drive_pulse[circuit]['Z']
                waveform_collection[wavename] = waveform_collection[wavename]+temp_pulse
                ## xy_mk 脉冲
                wavename = drive_info[circuit]['inst_source'] + '__' + str(drive_info[circuit]['mk_ch']) + '__mk'
                waveform_collection_flag[wavename] = waveform_collection_flag[wavename]+1
                temp_pulse = drive_pulse[circuit]['XY_mk']
                waveform_collection[wavename] = waveform_collection[wavename]+temp_pulse
            else:
                # couple 控制脉冲
                for couple in drive_pulse[circuit]:
                    wavename = config['couple_control_info'][couple]['inst_source'] + '__' + str(config['couple_control_info'][couple]['ch'])
                    waveform_collection_flag[wavename] = waveform_collection_flag[wavename]+1
                    temp_pulse = drive_pulse[circuit][couple]
                    waveform_collection[wavename] = waveform_collection[wavename]+temp_pulse
        
        ## 针对驱动结束后输出一个trigger给测量的情况，生成一个trigger 脉冲
        trig_delay = len(list(waveform_collection.values())[0].data)/sRate+config['awg_info']['meas_info']['meas_delay']
        trig_pulse = Blank(width=trig_delay,sRate=sRate)|DC(width=2e-6,sRate=sRate)
        trig_pulse = Wavedata.setLen(trig_pulse,pulse_length)
        ## 设置所有脉冲的总长度
        for wavename in waveform_collection:
            waveform_collection[wavename] = Wavedata.setLen(waveform_collection[wavename],pulse_length)
        
        ## 将waveform_collection处理成 Tek 5014 支持的形式
        waveform_for_awg = dict()
        for wavename in waveform_collection:
            source = wavename.split('__')[0]
            waveform_for_awg[source] = dict()
        # 先处理除 XY_mk 外的所有信号
        for wavename in waveform_collection:
            if len(wavename.split('__')) == 3:
                source,I_ch,Q_ch = wavename.split('__')
                if Q_ch != 'mk':
                    I_ch = 'ch'+str(I_ch)
                    Q_ch = 'ch'+str(Q_ch)
                    waveform_for_awg[source].update({I_ch: (Wavedata.I(waveform_collection[wavename]).data),
                                                     Q_ch: (Wavedata.Q(waveform_collection[wavename]).data)})
            else:
                source,I_ch = wavename.split('__')
                I_ch = 'ch'+str(I_ch)
                waveform_for_awg[source].update({I_ch: (Wavedata.I(waveform_collection[wavename]).data)})
        # 将 XY_mk 信号合并到对应 AWG 的通道上
        for wavename in waveform_collection:
            if len(wavename.split('__')) == 3:
                source,I_ch,Q_ch = wavename.split('__')
                if Q_ch == 'mk':
                    I_ch = 'ch'+str(I_ch)
                    mk = np.array(waveform_collection[wavename].data)
                    mk[mk>=1] = 1
                    mk[mk<1] = 0
                    mk = list(map(int,mk))
                    xy_wave = waveform_for_awg[source][I_ch]
                    waveform_for_awg[source].update({I_ch: (xy_wave,mk)})
        # 将给测量的trigger信号合并到对应AWG通道上
        mk = list(map(int,trig_pulse.data))
        source = config['awg_info']['meas_info']['meas_trig_source']
        I_ch = 'ch'+str(config['awg_info']['meas_info']['mess_trig_ch'])
        try:
            xy_wave = waveform_for_awg[source][I_ch]
        except KeyError:
            xy_wave = trig_pulse.data
        waveform_for_awg[source].update({I_ch: (xy_wave,mk)})
        ## 设置 AWG
        for source in waveform_for_awg:
            setting_dict = waveform_for_awg[source]
            ## 设置awg
            awg_app = qulab.make_app('AWG.awg_generate_waveform',parent=self).with_settings(setting_dict)
            await awg_app.done()
        yield waveform_for_awg

In [122]:
map_drive_sequence.save(package='base')

## map meas pulse

In [14]:
import qulab
import numpy as np
from qulab.tools.wavedata import *

class map_meas_pulse(qulab.Application):
    async def work(self):
        setting_dict = {'meas_pulse': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        meas_pulse = setting_dict['meas_pulse']
        
        meas_config_app = qulab.make_app('base.meas_config',parent=self)
        config = await meas_config_app.done()
        config = config[0]
        meas_info = config['awg_info']['meas_info']
        sRate = meas_info['sRate']
        pulse_length = meas_info['pulse_length']
        
        ## 初始化一个waveform_collection dict，其中key包好circuit、awg source 和 channel信息
        ## value即为对应通道的波形
        waveform_collection = dict()
        trig_collection = dict()
        waveform_collection_flag = dict()
        for circuit in meas_pulse:
            wavename = meas_info[circuit]['inst_source'] + '__' + str(meas_info[circuit]['I_ch'])+ '__' + str(meas_info[circuit]['Q_ch'])
            waveform_collection[wavename] = Wavedata(sRate=sRate)
            trig_collection[wavename] = Wavedata(sRate=sRate)
            waveform_collection_flag[wavename] = 0
        for circuit in meas_pulse:
            wavename = meas_info[circuit]['inst_source'] + '__' + str(meas_info[circuit]['I_ch'])+ '__' + str(meas_info[circuit]['Q_ch'])
            waveform_collection_flag[wavename] = waveform_collection_flag[wavename]+1
            # 若输入波形采样率和config里的采样率不一致，则按照config里的sRate重新采样
            if meas_pulse[circuit].sRate != sRate:
                meas_pulse[circuit] = meas_pulse[circuit].resample(sRate=sRate)
            # 根据trig_delay的符号，选择延迟测量脉冲或者延迟trigger脉冲
            if meas_info[circuit]['trig_delay'] < 0:
                temp_pulse = Blank(width=meas_info[circuit]['trig_delay'],sRate=sRate)|meas_pulse[circuit]
                trig_pulse = DC(width=len(meas_pulse[circuit])/sRate,sRate=sRate)
            else:
                temp_pulse = meas_pulse[circuit]
                trig_pulse = Blank(width=meas_info[circuit]['trig_delay'],sRate=sRate
                                  )|DC(width=len(meas_pulse[circuit].data)/sRate,sRate=sRate)
            # 相同source和channel信号进行叠加，并且处理因叠加而造成的offset偏差
            waveform_collection[wavename] = waveform_collection[wavename]+temp_pulse
            trig_collection[wavename] = trig_collection[wavename]+trig_pulse
            if waveform_collection_flag[wavename] > 1:
                I_offset = config['mw_info']['meas_info'][circuit]['IQ_cali_array'][0][1]
                Q_offset = config['mw_info']['meas_info'][circuit]['IQ_cali_array'][1][1]
                waveform_collection[wavename] = vIQmixer.carry_wave(carry_freq = 0, 
                                                                    IQ = waveform_collection[wavename], 
                                                                    carry_cali = [[1,-I_offset,0],[1,-Q_offset,0]],
                                                                    DEG = True)
                waveform_collection_flag[wavename] = 1
        for wavename in waveform_collection:
            waveform_collection[wavename] = Wavedata.setLen(waveform_collection[wavename],pulse_length)
            trig_collection[wavename] = Wavedata.setLen(trig_collection[wavename],pulse_length)
        
        waveform_for_awg = dict()
        for wavename in waveform_collection:
            source,I_ch,Q_ch = wavename.split('__')
            waveform_for_awg[source] = dict()
        for wavename in waveform_collection:
            source,I_ch,Q_ch = wavename.split('__')
            I_ch = 'ch'+str(I_ch)
            Q_ch = 'ch'+str(Q_ch)
            mk = np.array(trig_collection[wavename].data)
            mk[mk>=1] = 1
            mk[mk<1] = 0
            mk = list(map(int,mk))
            waveform_for_awg[source].update({I_ch: (Wavedata.I(waveform_collection[wavename]).data,mk),
                                             Q_ch: (Wavedata.Q(waveform_collection[wavename]).data,mk)})
        for source in waveform_for_awg:
            setting_dict = waveform_for_awg[source]
            ## 设置awg
            awg_app = qulab.make_app('AWG.awg_generate_waveform',parent=self).with_settings(setting_dict)
            await awg_app.done()
        yield 'ok'

In [15]:
map_meas_pulse.save(package='base')

# app for instrument

## ATS

### ATS_acquire

In [16]:
import numpy as np
import asyncio
import qulab

class ats_acquire(qulab.Application):
    '''采集卡进行一次数据采集操作
    require:
        rc : ATS
        settings: repeat(optional)
    return: data
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'n': 1024, 
                        'ARange': 0.1, 
                        'BRange': 0.1, 
                        'repeats': 2048., 
                        'maxlen': 2048,
                        'bufferCount': 2048,
                        'trigLevel': 0.5,
                        'trigDelay': 0,
                        'trigTimeout': 0,
                        'average':True}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
                
        ats = self.rc['ATS']
        ats.performOpen()
        ats.set(**setting_dict)
        A, B = ats.getTraces(avg=setting_dict['average'])
        yield A, B
    """
    @staticmethod
    def plot(fig, data):
        I, Q = data
        ax = fig.add_subplot(111)
        ax.plot(I, 'ro-')
        ax.plot(Q, 'b*-')
        ax.set_xlabel('time (ns)')
        ax.set_ylabel('amp (a.u.)')
    """

In [17]:
ats_acquire.save(package='ATS')

### ATS result process

In [ ]:
import numpy as np
from scipy.fftpack import fft,ifft
from qulab.tools.wavedata import *
import qulab

class ats_result_process(qulab.Application):
    async def work(self):
        setting_dict = {'I': None, 
                        'Q': None,
                        'meas_info':dict(),
                        'qubit_info':dict(),
                        'measure_qubit_list': ['Qubit1','Qubit2'],
                        'sRate': 1e9,
                        'ats_repeats': 1}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        meas_info = setting_dict['meas_info']
        qubit = setting_dict['qubit_info']
        
        # ats_repeats大于1表示进行single shot测试
        if setting_dict['ats_repeats']>1:
            qubit_I = []
            qubit_Q = []
            target_idx = 0
            center_array = []
            IQ_array = []
            s_flag = np.zeros([len(setting_dict['measure_qubit_list']),len(setting_dict['I'])])
            for target in setting_dict['measure_qubit_list']:
                single_shot_array = qubit[target]['single_shot_array']
                f = abs(float(meas_info[target]['carry']))
                ### 定义滤波器
                IF_filter0 = F.IIRFilter(N=2, Wn=[f-5e6, f+5e6], 
                                        rp=0.01, rs=100, btype='band', ftype='ellip', fs=1e9)
                IF_filter1 = F.IIRFilter(N=2, Wn=[f-0.5e6, f+0.5e6], 
                                        rp=0.01, rs=100, btype='band', ftype='ellip', fs=1e9)
                #取数据
                chA=np.array(setting_dict['I'])
                chB=np.array(setting_dict['Q'])
                # 对每一个测量处理得到IQ值
                qubit_I0 = []
                qubit_Q0 = []
                for idx in range(int(setting_dict['ats_repeats'])):
                    I = chA[idx]
                    Q = chB[idx]
                    wd_raw = Wavedata(I+1j*Q,sRate=1e9).filter(IF_filter0)
                    wd_raw = wd_raw.filter(IF_filter1)
                    iqcali = A.Analyze_cali(wd_raw, f)
                    wd_f = A.Homodyne(wd_raw, freq=f, cali=iqcali)
                    amp_I = np.mean(Wavedata.I(wd_f).data[50:-50])
                    amp_Q = np.mean(Wavedata.Q(wd_f).data[50:-50])
                    qubit_I0.append(amp_I)
                    qubit_Q0.append(amp_Q)
                qubit_I0 = np.array(qubit_I0)
                qubit_Q0 = np.array(qubit_Q0)
                IQ_array.append([qubit_I0,qubit_Q0])
                # 记录原始分布的中心
                center_array.append([np.mean(qubit_I0),np.mean(qubit_Q0)])

                # 计算2个状态的概率
                s_0_d = (qubit_I0-single_shot_array[0])**2+(qubit_Q0-single_shot_array[1])**2
                s_1_d = (qubit_I0-single_shot_array[2])**2+(qubit_Q0-single_shot_array[3])**2
                # |0>、|1>概率
                s_flag[target_idx][1.4*s_0_d>s_1_d] = 2**target_idx
                target_idx += 1
            # 计算各个状态的概率
            p_flag = np.sum(s_flag,axis=0)
            probility_array = []
            for idx in range(2**len(setting_dict['measure_qubit_list'])):
                probility_array.append(len(np.where(p_flag==idx)[0])/len(p_flag))
            probility_array = np.array(probility_array)
            yield IQ_array, probility_array, center_array
        else:        
            I = setting_dict['I'][50:]
            Q = setting_dict['Q'][50:]
                
            for target in setting_dict['measure_qubit_list']:
                ### 定义滤波器，并绘制响应曲线
                f = abs(float(meas_info[target]['carry']))
                IF_filter0 = F.IIRFilter(N=2, Wn=[f-5e6, f+5e6], 
                                        rp=0.01, rs=100, btype='band', ftype='ellip', fs=1e9)
                IF_filter1 = F.IIRFilter(N=2, Wn=[f-0.5e6, f+0.5e6], 
                                        rp=0.01, rs=100, btype='band', ftype='ellip', fs=1e9)
                wd_raw=Wavedata(I+1j*Q,setting_dict['sRate']).filter(IF_filter0)
                wd_raw=wd_raw.filter(IF_filter1)
                iqcali = A.Analyze_cali(wd_raw, f)           
                wd_f = A.Homodyne(wd_raw, freq=f, cali=iqcali)
                amp_I = np.mean(Wavedata.I(wd_f).data[50:-50])
                amp_Q = np.mean(Wavedata.Q(wd_f).data[50:-50])
                # 幅度归一化
                amp = np.sqrt(amp_I**2+amp_Q**2)
                amp = (amp-qubit[target]['readout_base'])/qubit[target]['rabi_amp']+1
                yield amp, amp_I, amp_Q

In [19]:
ats_result_process.save(package='ATS')


## MW

### MW control base

In [20]:
import numpy as np
import qulab

class mw_control_base(qulab.Application):
    '''
    对两台MW进行功率和频率设置
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'drive_mw_source': None, 
                        'meas_mw_source': None, 
                        'drive_freq': None, 
                        'meas_freq': None,
                        'drive_power': None, 
                        'meas_power': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 设置MW
        if setting_dict['drive_mw_source']:
            drive_mw = qulab.open_resource(setting_dict['drive_mw_source'])
            if setting_dict['drive_power']:
                drive_mw.setValue('Power', setting_dict['drive_power'])
            if setting_dict['drive_freq']:
                drive_mw.setValue('Frequency', setting_dict['drive_freq'])
            drive_mw.setValue('Output', 'ON')
        if setting_dict['meas_mw_source']:
            meas_mw = qulab.open_resource(setting_dict['meas_mw_source'])
            if setting_dict['meas_power']:
                meas_mw.setValue('Power', setting_dict['meas_power'])
            if setting_dict['meas_freq']:
                meas_mw.setValue('Frequency', setting_dict['meas_freq'])
            meas_mw.setValue('Output', 'ON')
        yield 'OK'

In [21]:
mw_control_base.save(package='MW')

### set MW

In [22]:
import qulab

class set_mw(qulab.Application):
    async def work(self):
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'set_drive': True,
                        'set_meas': True}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # mw_info从外部输入，以便需要扫描mw_info中某个参数的app中调用此app,若未输入则从config_info内获取
        mw_info = setting_dict['mw_info']
        if not mw_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            mw_info = config['mw_info']
        
        drive_source = list()
        meas_source = list()
        # 对circuit_list内所有circuit都处理一遍，但若mw source是重复的，则跳过不再处理
        for circuit in setting_dict['circuit_list']:
            source = mw_info['drive_info'][circuit]['inst_source']
            if source in drive_source or not setting_dict['set_drive']:
                pass
            else:
                drive = {'source': source,
                         'power': mw_info['drive_info'][circuit]['power'],
                         'frequency': mw_info['drive_info'][circuit]['frequency']}
                mw = qulab.open_resource(drive['source'])
                mw.setValue('Power', drive['power'])
                mw.setValue('Frequency', drive['frequency'])
                mw.setValue('Output', 'ON')
                drive_source.append(source)
            
            source = mw_info['meas_info'][circuit]['inst_source']
            if source in meas_source or not setting_dict['set_drive']:
                pass
            else:
                meas = {'source': source,
                        'power': mw_info['meas_info'][circuit]['power'],
                        'frequency': mw_info['meas_info'][circuit]['frequency']}
                mw = qulab.open_resource(meas['source'])
                mw.setValue('Power', meas['power'])
                mw.setValue('Frequency', meas['frequency'])
                mw.setValue('Output', 'ON')
                meas_source.append(source)
            
            mw_control[circuit] = {'drive': drive,
                                    'meas': meas}
        yield mw_control

In [23]:
set_mw.save(package='MW')

### MW sweep one parameter

In [24]:
import numpy as np
import asyncio
import qulab

class mw_sweep_one_para(qulab.Application):
    '''
    对两台MW进行功率和频率扫描
    可选为meas_mw_freq、meas_mw_power、drive_mw_freq、drive_mw_power
    输入格式为
    app.sweep([('para_x', ***)])
    在with_settings中的sweep_para说明para_x物理上对应上述4组中的哪组
    利用采集卡进行数据采集
    通过该app可以构建S21参数（2D和3D）扫描、qubit能谱测试、单比特时域测试等
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'sweep_para': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # 获取config_info
        mw_info = setting_dict['mw_info']
        if not mw_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            mw_info = config['mw_info']     
        ## 设置MW, 扫描参数进行测量
        if isinstance(setting_dict['sweep_para'],list):
            sweep_para = setting_dict['sweep_para'][0]
        else:
            sweep_para = setting_dict['sweep_para']
        async for x in self.sweep['para_x']:
            for circuit in setting_dict['circuit_list']:
                if 'drive' in sweep_para and 'power' in sweep_para:
                    mw_info['drive_info'][circuit]['power'] = x
                elif 'drive' in sweep_para and 'freq' in sweep_para:
                    mw_info['drive_info'][circuit]['frequency'] = x
                elif 'meas' in sweep_para and 'power' in sweep_para:
                    mw_info['meas_info'][circuit]['power'] = x
                elif 'meas' in sweep_para and 'freq' in sweep_para:
                    mw_info['meas_info'][circuit]['frequency'] = x
            setting_dict[sweep_para] = x
            set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings({'circuit_list': setting_dict['circuit_list'],
                                                                                'mw_info': mw_info})
            await set_mw_app.done()
            
            if 'ATS' in setting_dict['child_app']:
                ## 采集卡进行一次数据采集
                ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                I, Q = await ats_app.done()
                if setting_dict['result_process_app'] is None:
                    time = np.linspace(1,len(I),len(I))
                    yield x, time, I, Q, sweep_para
                else:
                    time = 0
                    setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                    process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                                ).with_settings(setting_dict['result_process_app_setting'])
                    result = await process_app.done()
                    if isinstance(result[0], np.ndarray):
                        yield x, time, result[0][0], result[1][0], sweep_para
                    else:
                        yield x, time, result[0], result[1], sweep_para
            elif 'PNA' in setting_dict['child_app']:
                ## PNA进行一次测量
                pna_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                freq, re, im, amp, pha = await pna_app.done()
                yield x, freq, amp, pha, sweep_para
            else:
                print('This child app is not supported')
                break
            
    async def set_para_x(self, x):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, time, I, Q, sweep_para):
        if  self.data.rows> 1:
            sweep_para = sweep_para[0]
            time = time[0]
        return x, time, I, Q, sweep_para
                  
    @staticmethod
    def plot(fig, data):
        x, y, amp, pha, sweep_para = data
        #if type(y) is list: 
        if isinstance(y, np.ndarray) and len(y)>1:
            ax1 = fig.add_subplot(121)
            ax2 = fig.add_subplot(122)
            if y[-1]>1e6:
                ratio = 1e9
            else:
                ratio = 1
            if isinstance(x, np.ndarray):
                a = ax1.imshow(amp.T, extent=(min(x), max(x), min(y)/ratio, max(y)/ratio),
                               aspect='auto', origin='lower', interpolation='nearest')
                ax2.imshow(pha.T, extent=(min(x), max(x), min(y)/ratio, max(y)/ratio),
                           aspect='auto', origin='lower', interpolation='nearest')
                if 'power' in sweep_para:
                    ax1.set_xlabel('%s (dBm)'%(sweep_para))
                    ax2.set_xlabel('%s (dBm)'%(sweep_para))
                    ax1.set_ylabel('freq(GHz) / time(ns)')
                    ax2.set_ylabel('freq(GHz) / time(ns)')
                else:
                    ax1.set_xlabel('%s (Hz)'%(sweep_para))
                    ax2.set_xlabel('%s (Hz)'%(sweep_para))
                    ax1.set_ylabel('freq(GHz) / time(ns)')
                    ax2.set_ylabel('freq(GHz) / time(ns)')
                fig.colorbar(a)
            else:
                ax1.plot(y/ratio, amp)
                ax2.plot(y/ratio, pha)
                ax1.set_xlabel('freq(GHz) / time(ns)')
                ax2.set_xlabel('freq(GHz) / time(ns)')
                ax1.set_ylabel('S21(dB) / amp(a.u)')
                ax2.set_ylabel('pha(degree) / amp(a.u)')
            ax1.set_title('Amplitude / I',fontsize=15)
            ax2.set_title('Phase / Q',fontsize=15)
        else:
            ax1 = fig.add_subplot(121)
            ax2 = fig.add_subplot(122)
            ax1.plot(x, amp)
            ax2.plot(x, pha)
            if 'power' in sweep_para:
                ax1.set_xlabel('%s (dBm)'%(sweep_para))
                ax2.set_xlabel('%s (dBm)'%(sweep_para))
            else:
                ax1.set_xlabel('%s (Hz)'%(sweep_para))
                ax2.set_xlabel('%s (Hz)'%(sweep_para))
            ax1.set_ylabel('S21(dB) / amp(a.u)')
            ax2.set_ylabel('pha(degree) / amp(a.u)')
            ax1.set_title('Amplitude / I',fontsize=15)
            ax2.set_title('Phase / Q',fontsize=15)

In [25]:
mw_sweep_one_para.save(package='MW')

### MW sweep two parameters

In [26]:
import numpy as np
import asyncio
import qulab

class mw_sweep_two_para(qulab.Application):
    '''
    以mw_sweep_one_para.app为基础，增加一维扫描参数
    对两台MW进行功率和频率扫描
    可选为meas_mw_freq、meas_mw_power、drive_mw_freq、drive_mw_power
    输入格式为
    app.sweep([('para_x', ***),('para_y', ***)])
    在with_settings中的sweep_para说明para_x物理上对应上述4组中的哪2组
    如‘sweep_para':['meas_mw_freq','meas_mw_power']
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'sweep_para': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # 获取config_info
        mw_info = setting_dict['mw_info']
        if not setting_dict['mw_info']:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            setting_dict['mw_info'] = config['mw_info']
        ## 临时记录需要进行扫描的para
        para_x = setting_dict['sweep_para'][0]
        para_y = setting_dict['sweep_para'][1]
        # 更新sweep_para为仅保留para_x，以便给mw_sweep_one_para.app
        setting_dict['sweep_para'] = para_x
        ## 扫描para_y
        async for y in self.sweep['para_y']:
            for circuit in setting_dict['circuit_list']:
                if 'drive' in sweep_para and 'power' in para_y:
                    mw_info['drive_info'][circuit]['power'] = y
                elif 'drive' in sweep_para and 'freq' in para_y:
                    mw_info['drive_info'][circuit]['frequency'] = y
                elif 'meas' in sweep_para and 'power' in para_y:
                    mw_info['meas_info'][circuit]['power'] = y
                elif 'meas' in sweep_para and 'freq' in para_y:
                    mw_info['meas_info'][circuit]['frequency'] = y
            mw_app_one_para = qulab.make_app('MW.mw_sweep_one_para',parent=self).sweep(
                [('para_x',self.sweep['para_x'])]
            ).with_settings(setting_dict)
            x, freq, amp, pha, para_x = await mw_app_one_para.done()
            sweep_para = {'para_x':para_x,'para_y':para_y}
            yield x, y, amp, pha, sweep_para
        
    async def set_para_y(self, y):
        await asyncio.sleep(0.002)
        
    def pre_save(self, x, y, amp, pha, sweep_para):
        if self.data.rows > 1:
            x = x[0]
            sweep_para = sweep_para[0]
        return x, y, amp, pha, sweep_para
        
    @staticmethod
    def plot(fig, data):
        x, y, amp, pha, sweep_para = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if 'freq' in sweep_para['para_x']:
            x_plot_ratio = 1e-9
            x_plot_label = 'GHz'
        else:
            x_plot_ratio = 1
            x_plot_label = 'dBm'
        if 'freq' in sweep_para['para_y']:
            y_plot_ratio = 1e-9
            y_plot_label = 'GHz'
        else:
            y_plot_ratio = 1
            y_plot_label = 'dBm'
        if isinstance(y, np.ndarray):
            a = ax1.imshow(amp, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                        min(y)*y_plot_ratio, max(y)*y_plot_ratio),
                           aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                    min(y)*y_plot_ratio, max(y)*y_plot_ratio),
                       aspect='auto', origin='lower', interpolation='nearest')
            fig.colorbar(a)
        else:
            ax1.plot(x*x_plot_ratio, amp)
            ax2.plot(x*x_plot_ratio, pha)
        ax1.set_xlabel('%s (%s)'%(sweep_para['para_x'],x_plot_label))
        ax1.set_ylabel('%s (%s)'%(sweep_para['para_y'],y_plot_label))
        ax2.set_xlabel('%s (%s)'%(sweep_para['para_x'],x_plot_label))
        ax2.set_ylabel('%s (%s)'%(sweep_para['para_y'],y_plot_label))

In [27]:
mw_sweep_two_para.save(package='MW')

## PNA

### PNA S21

In [28]:
import numpy as np
import skrf as rf
import qulab

class S21(qulab.Application):
    '''从网分上读取 S21
    require:
        rc : PNA
        settings: 以dict形式输入，若未给则按照PNA当前状态进行数据采集
    return: Frequency, Re(S21), Im(S21), amp, pha
    '''
    async def work(self):
        pna = self.rc['PNA']
        ## 取测试相关参数和设备信息
        setting_dict = {'repeat': 1,
                        'Power': -30,
                        'Frequency center': 5.0e9,
                        'Frequency span': 100e6,
                        'Number of points': 101,
                        'Bandwidth': 50,
                        'Output': 'ON',
                        'digital_delay': 10e-9,
                        'phase_compensate': True,
                        'timeout': 10}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
                pna.setValue(setting, setting_dict[setting])
        pna.set_timeout(setting_dict['timeout'])
        ## 获取设置好的frequency
        x = np.array(self.rc['PNA'].get_Frequency())
        pha_offset = (x*setting_dict['digital_delay']*360+180)%360 - 180
        ## 进行S21采集
        for i in range(setting_dict['repeat']):
            self.processToChange(100.0 / setting_dict['repeat'])
            y = np.array(self.rc['PNA'].get_S())
            amp = rf.mag_2_db(np.abs(y))
            pha = np.arctan(np.imag(y)/np.real(y))*180/np.pi
            if setting_dict['phase_compensate']:
                pha = list(map(self.get_phase_compensate, y, pha, pha_offset))
            yield x, np.real(y), np.imag(y), amp, np.array(pha)
            self.increaseProcess()
            
    def pre_save(self, x, re, im, amp, pha):
        if self.data.rows > 1:
            x = x[0]
            re = np.mean(re, axis=0)
            im = np.mean(im, axis=0)
            amp = np.mean(amp, axis=0)
            pha = np.mean(pha, axis=0)
        return x, re, im, amp, pha
    
    @staticmethod
    def get_phase_compensate(s, pha, offset):
        if np.real(s)<0:
            if pha<0:
                pha = 180 + pha
            else:
                pha = pha - 180
        pha = pha+offset
        if pha>180:
            pha =pha-360
        if pha<-180:
            pha =pha+360
        return pha

    @staticmethod
    def plot(fig, data):
        x, re, im, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x/1e9, amp)
        ax2.plot(x/1e9, pha)
        ax1.set_xlabel('Frequency / GHz')
        ax1.set_ylabel('S21 / dB')
        ax2.set_xlabel('Frequency / GHz')
        ax2.set_ylabel('Phase / degree')

In [29]:
S21.save(package='PNA')

## AWG

### AWG generate waveform

In [30]:
import numpy as np
import asyncio
import re
import qulab
from qulab.tools.wavedata import *

class awg_generate_waveform(qulab.Application):
    '''
    输入波形列表，awg输出波形
    采样率固定为1e9，参考时钟为外部
    输入参数：
    app.with_rc({'AWG':'XXXX'}).with_settings({'ch0': ch0_waveform, 'ch1': ch1_waveform})
    波形名称格式为'chx',其中'x'为数字，且只能为1位，’x'对应即为信号输出通道
    波形数据，例如ch1_waveform为2-D array,第1维对应通道模拟信号数据，第2维对应为该通道的marker数据
    with_settings()可以包含其他对awg的设置
    '''
    async def work(self):
        awg = self.rc['AWG']
        awg.stop()
        ## 取测试相关参数和设备信息
        setting_dict = {'Sample Rate': 1.2e9, 
                        'Clock source': 'Internal',
                        'Reference Source': 'External',
                        'Run Mode': 'Triggered',
                        'ch0': None,
                        'ch1':None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
                awg.setValue(setting, setting_dict[setting])
        pattern = re.compile(r'ch\d')
        for x in self.settings:
            m = pattern.match(x)
            if m:
                wavename='wave'+str(x)
                waveform=self.settings.get(x,0)
                if len(waveform)==3:
                    awg.remove_waveform(name=wavename)
                    awg.create_waveform(name=wavename, length=len(waveform[0]))
                    awg.update_waveform(points=waveform[0], name=wavename)
                    awg.update_marker(name=wavename, mk1=waveform[1], mk2=waveform[2])
                elif len(waveform)==2:
                    awg.remove_waveform(name=wavename)
                    awg.create_waveform(name=wavename, length=len(waveform[0]))
                    awg.update_waveform(points=waveform[0], name=wavename)
                    awg.update_marker(name=wavename, mk1=waveform[1])
                else:
                    awg.remove_waveform(name=wavename)
                    awg.create_waveform(name=wavename, length=len(waveform))
                    awg.update_waveform(points=waveform, name=wavename)
                awg.use_waveform(name=wavename, ch=int(x[2]))
                awg.setValue('Vpp', 2.0, ch=int(x[2]))
                awg.output_on(ch=int(x[2]))
                await asyncio.sleep(0.1)
        awg.run()
        yield 'ok'

In [31]:
awg_generate_waveform.save(package='AWG')

## RSA

### RSA spectrum line

In [32]:
import numpy as np
import qulab

class spectrum_line(qulab.Application):
    '''从Tek_RSA5126B spectrum analyzer读取一条频谱线
    '''
    async def work(self):
        RSA = self.rc['RSA']
        ## 取测试相关参数和设备信息
        setting_dict = {'Add_general_measurement': ['Spectrum',],
                        'Frequency center': 5.0e9,
                        'Frequency span': 10e6,
                        'Points': '2401',
                        'Bandwidth': 50,
                        'Spectrum_length': 0.1}
        for setting in setting_dict:
            if self.settings.get(setting, None):
                setting_dict[setting] = self.settings.get(setting, 0)
            if setting == 'Add_general_measurement':
                for x in setting_dict[setting]:
                    RSA.setValue(setting, x)
            else:
                RSA.setValue(setting, setting_dict[setting])
        ## 进行spectrum trace采集
        spectrum = RSA.get_spectrum_trace()
        yield spectrum[0], spectrum[1]

    def pre_save(self, x, y):
        if self.data.rows > 1:
            x = x[0]
        return x, y

    @staticmethod
    def plot(fig, data):
        x, y = data
        ax = fig.add_subplot(111)
        ax.plot(x/1e9, y)
        ax.set_xlabel('Frequency (GHz)')
        ax.set_ylabel('Power (dBm)')

In [33]:
spectrum_line.save(package='RSA')

## GS200

### GS200 control base

In [34]:
import qulab

class GS200_control_base(qulab.Application):
    '''
    对GS200进行设置
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'GS200_source': 'GS200', 
                        'Mode': 'current', 
                        'Range': '0.01', 
                        'Level': 0.001,
                        'ProtectCurr': 1.5, 
                        'ProtectVolt': 5,
                        'Output': 'ON'}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # 进行参数配置
        GS200 = qulab.open_resource(setting_dict['GS200_source'])
        for setting in setting_dict:
            GS200.setValue(setting, setting_dict[setting])
        yield 'OK'

In [35]:
GS200_control_base.save(package='GS200')

## UHF

### UHF generate waveform

In [36]:
import numpy as np
import asyncio
import re
import qulab
from qulab.tools.wavedata import *

class uhfawg_generate_waveform(qulab.Application):
    '''
    输入波形列表，awg输出波形
    采样率固定为1.8e9，参考时钟为外部
    输入参数：
    app.with_rc({'AWG':'XXXX'}).with_settings({'ch0': ch0_waveform, 'ch1': ch1_waveform})
    波形名称格式为'chx',其中'x'为数字，且只能为1位，’x'对应即为信号输出通道
    波形数据，例如ch0_waveform为1-D array,对应通道模拟信号数据
    with_settings()可以包含其他对awg的设置
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'SampleRate': '900M',
                        'Imp50': 'ON',
                        'MaxRange': '0.75',
                        'RunMode': 'Continuous',
                        'TrigSlope': 'Rise',
                        'Output': 'ON',
                        'TrigChannel': 'TrigIn1',
                        'Modulation': 'Plain',
                        'aux_out': False,
                        'ch0': None,
                        'ch1': None,
                        'aux_ch0': None,
                        'aux_ch1': None,
                        'aux_ch2': None,
                        'aux_ch3': None}
        awg = self.rc['AWG']
        awg.aux_out = self.settings.get('aux_out')
        pattern = re.compile(r'ch\d')
        aux_pattern = re.compile(r'aux_ch\d')
        init_flag = False
        for setting in self.settings:
            if setting in setting_dict:
                m = pattern.match(setting)
                m_aux = aux_pattern.match(setting)
                if m==None and m_aux==None:
                    setting_dict[setting] = self.settings.get(setting, 1)
                elif m:
                    m = None
                    wavename='wave'+str(setting)
                    waveform=self.settings.get(setting,1)
                    awg.update_waveform(points=waveform, name=wavename)
                    # 若AWG内已经存在波形，且波形长度和当前要设置的波形长度相等，
                    # 则不需要create_waveform，只需要update_waveform
                    if not init_flag:
                        waveform_node = '/%s/awgs/0/waveform/waves/0' % (awg.device)
                        try:
                            d = awg.daq.get(waveform_node,True)
                            data = next(iter(d.values()))
                            data_temp = []
                            data_temp = data[0]['vector']
                            if (len(data_temp)!=len(waveform)*2):
                                awg.init()
                                init_flag = True
                        except TypeError:
                            awg.init()
                            init_flag = True 
                else:
                    m_aux = None
                    if awg.aux_out:
                        aux_wavename='wave'+str(setting)
                        aux_waveform=self.settings.get(setting,1)
                        awg.update_waveform(points=aux_waveform, name=aux_wavename)
                        # 若AWG内已经存在波形，且波形长度和当前要设置的波形长度相等，
                        # 则不需要create_waveform，只需要update_waveform
                        if not init_flag:
                            waveform_node = '/%s/awgs/0/waveform/waves/1' % (awg.device)
                            try:
                                d = awg.daq.get(waveform_node,True)
                                data = next(iter(d.values()))
                                data_temp = []
                                data_temp = data[0]['vector']
                                if (len(data_temp)!=len(aux_waveform)*4):
                                    awg.init()
                                    init_flag = True
                            except TypeError:
                                awg.init()
                                init_flag = True
        # 为了避免在create_waveform中将设备init导致设置被初始化，setValue放在了这里
        for setting in setting_dict:
            awg.setValue(setting, setting_dict[setting], ch=0)
            awg.setValue(setting, setting_dict[setting], ch=1)
        if setting_dict['aux_out'] and init_flag:
            awg.create_waveform(length=len(waveform), aux_length=len(aux_waveform))
        elif init_flag:
            awg.create_waveform(length=len(waveform))
        # 输出波形
        if setting_dict['aux_out']:
            awg.use_waveform(name=['wavech0','wavech1',
                                   'waveaux_ch0','waveaux_ch1',
                                   'waveaux_ch2','waveaux_ch3',])
        else:
            awg.use_waveform(name=['wavech0','wavech1',])
        yield waveform
    
    @staticmethod
    def plot(fig, waveform):
        ax = fig.add_subplot(111)
        ax.plot(waveform[0], 'r')
        ax.plot(waveform[1], 'b')

In [37]:
uhfawg_generate_waveform.save(package='UHF')

# JPA

## find jpa working point

In [39]:
import qulab
import asyncio
import skrf as rf
import numpy as np


class find_jpa_working_point(qulab.Application):
    """
    扫描jpa偏置和驱动功率，测量最佳工作点
    """
    async def work(self):
        setting_dict = {'bias_list': None,
                        'power_list': None}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        bias_list = setting_dict['bias_list']
        power_list = setting_dict['power_list']
        # 获取config_info
        meas_config_app = qulab.make_app('base.meas_config',parent=self)
        config = await meas_config_app.done()
        config = config[0]
        jpa_info = config['jpa_info']
        
        #设置PNA
        pna = qulab.open_resource(config['inst_info']['pna'])
        pna_setting = config['pna_info']
        pna_setting.update({'Frequency center': pna_setting['center'],
                            'Frequency span': pna_setting['span'],
                            'Number of points': pna_setting['points'],})
        app = qulab.make_app('PNA.S21',parent=self).with_rc({'PNA':config['inst_info']['pna']}
                                               ).with_settings(pna_setting)
        data = await app.done()
        
        #设置驱动微波源
        mw = qulab.open_resource(jpa_info['drive_mw'])
        mw.setValue('Frequency',jpa_info['drive_freq'])
        mw.setValue('Output','ON')
        
        #设置bias驱动源
        dc = qulab.open_resource(jpa_info['bias_inst'])
        
        # 进行测试
        for bias in bias_list:
            dc.setValue('Offset',bias,ch=jpa_info['bias_ch'])
            await asyncio.sleep(0.2)
            amp_list = []
            pha_list = []
            for power in power_list:
                mw.setValue('Power',power)
                await asyncio.sleep(0.01)
                y = np.array(pna.get_S())
                amp = rf.mag_2_db(np.abs(y))
                pha = np.arctan(np.imag(y)/np.real(y))*180/np.pi
                amp_list.append(np.mean(amp))
                pha_list.append(np.mean(pha))
            amp_list = np.array(amp_list)
            pha_list = np.array(pha_list)
            yield power_list, bias, amp_list, pha_list
        
    def pre_save(self, power_list, bias, amp_list, pha_list):
        if self.data.rows > 1:
            power_list = power_list[0]
        return power_list, bias, amp_list, pha_list
            
    @staticmethod
    def plot(fig, data):
        power_list, bias, amp_list, pha_list = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if isinstance(bias, np.ndarray):
            a = ax1.imshow(amp_list, extent=(min(power_list), max(power_list), min(bias), max(bias)),
                          aspect='auto', origin='lower', interpolation='nearest')
            fig.colorbar(a)
            b = ax2.imshow(pha_list, extent=(min(power_list), max(power_list), min(bias), max(bias)),
                          aspect='auto', origin='lower', interpolation='nearest')
            fig.colorbar(b)
        else:
            ax1.plot(power_list,amp_list,'r-*')
            ax2.plot(power_list,pha_list,'r-*')
        ax1.set_title('amp',fontsize=15)
        ax2.set_title('pha',fontsize=15)
        ax1.set_ylabel('bias')
        ax1.set_xlabel('power') 
        ax2.set_xlabel('power') 

In [40]:
find_jpa_working_point.save(package='jpa')

# single bit characterize

## 3D S21 sweep meas amp

### with PNA

In [41]:
import numpy as np
import skrf as rf
import asyncio
import qulab

class three_dim_S21_with_PNA(qulab.Application):
    '''扫PNA功率进行cavity shift试验'''
    async def work(self):
        pna = self.rc['PNA']
        ## 取测试相关参数和设备信息
        setting_dict = {'Power': -30,
                        'Frequency center': 5.0e9,
                        'Frequency span': 100e6,
                        'Number of points': 101,
                        'Bandwidth': 50,
                        'Output': 'ON',
                        'digital_delay': 10e-9,
                        'phase_compensate': True,
                        'timeout': 1000}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
                pna.setValue(setting, setting_dict[setting])
            
        async for y in self.sweep['power']:
            # 一定要注意设置 parent
            setting_dict['Power']=y
            S21_app = qulab.make_app('PNA.S21', parent=self).with_settings(setting_dict)
            freq, re, im, amp, pha = await S21_app.done()
            yield freq, y, amp, pha

    async def set_power(self, y):
        await asyncio.sleep(0.001)
    
    def pre_save(self, freq, y, amp, pha):
        if self.data.rows > 1:
            freq = freq[0]
        return freq, y, amp, pha

    @staticmethod
    def plot(fig, data):
        x, y, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if isinstance(y, np.ndarray):
            ax1.imshow(amp, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                       aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                       aspect='auto', origin='lower', interpolation='nearest')
        else:
            ax1.plot(x/1e9, amp)
            ax2.plot(x/1e9, pha)
        ax1.set_title('Amplitude',fontsize=15)
        ax2.set_title('Phase',fontsize=15)
        ax1.set_ylabel('power (dBm)')
        ax1.set_xlabel('frequency (GHz)')
        ax2.set_ylabel('power (dBm)')
        ax2.set_xlabel('frequency (GHz)')

In [42]:
three_dim_S21_with_PNA.save(package='PNA')

### with ATS

In [43]:
import numpy as np
import asyncio
import qulab
from qulab.tools.wavedata import *

class three_dim_S21_with_ATS_sweep_awg_amp(qulab.Application):
    '''扫功率进行cavity shift试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        # 获取config_info
        meas_config_app = qulab.make_app('base.meas_config',parent=self)
        config = await meas_config_app.done()
        config = config[0]
        meas_info = config['mw_info']['meas_info']
        qubit_info = config['qubit_info']
        
        async for y in self.sweep['para_y']:
            # 生成meas_pulse
            qubit_info[setting_dict['circuit_list'][0]]['probe_amp'] = y
            meas_pulse_app = qulab.make_app('base.meas_pulse_generate',parent=self
                                           ).with_settings({'circuit_list':setting_dict['circuit_list'],
                                                            'meas_info': meas_info,
                                                            'qubit_info': qubit_info})
            meas_pulse = await meas_pulse_app.done()
            meas_pulse = meas_pulse[0]
            set_meas_awg_app = qulab.make_app('base.map_meas_pulse',parent=self).with_settings({'meas_pulse': meas_pulse})
            state = await set_meas_awg_app.done()
            
            # 扫描频率，测S21
            setting_dict['mw_info'] = config['mw_info']
            setting_dict['sweep_para'] = ['meas_freq',]
            app = qulab.make_app('MW.mw_sweep_one_para',parent=self).sweep([
                ('para_x', self.sweep['para_x'])]
            ).with_settings(setting_dict)
            x, freq, amp, pha, para_x = await app.done()
            yield x, y, amp, pha

    async def set_para_y(self, y):
        await asyncio.sleep(0.02)
    
    def pre_save(self, x, y, amp, pha):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp, pha

    @staticmethod
    def plot(fig, data):
        x, y, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        x_plot_ratio = 1e-9
        x_plot_label = 'GHz'
        if isinstance(y, np.ndarray):
            a = ax1.imshow(amp, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                        min(y), max(y)),
                           aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                    min(y), max(y)),
                       aspect='auto', origin='lower', interpolation='nearest')
            fig.colorbar(a)
        else:
            ax1.plot(x*x_plot_ratio, amp)
            ax2.plot(x*x_plot_ratio, pha)
        ax1.set_title('Amplitude',fontsize=15)
        ax2.set_title('Phase',fontsize=15)
        ax1.set_xlabel('freq (GHz))')
        ax1.set_ylabel('amp (a.u)')
        ax2.set_xlabel('freq (GHz))')
        ax2.set_ylabel('amp (a.u)')

In [44]:
three_dim_S21_with_ATS_sweep_awg_amp.save(package='ATS')

### with ATT

In [45]:
import numpy as np
import skrf as rf
import asyncio
import qulab

class three_dim_S21_with_ATS_and_ATT(qulab.Application):
    '''扫功率进行cavity shift试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'child_app': 'ATS_.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        # 获取config_info
        meas_config_app = qulab.make_app('base.meas_config',parent=self)
        config = await meas_config_app.done()
        config = config[0]
        setting_dict['mw_info'] = config['mw_info']
        setting_dict['sweep_para'] = ['meas_freq',]
        
        att = qulab.open_resource(config['inst_info']['att'])
        async for y in self.sweep['para_y']:
            # 一定要注意设置 parent
            att.setValue('Att', y)
            app = qulab.make_app('MW.mw_sweep_one_para',parent=self).with_rc({'ATS':'ATS'}).sweep([
                ('para_x', self.sweep['para_x'])]
            ).with_settings(setting_dict)
            x, freq, amp, pha, para_x = await app.done()
            yield x, y, amp, pha

    async def set_para_y(self, y):
        await asyncio.sleep(0.002)
    
    def pre_save(self, x, y, amp, pha):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp, pha

    @staticmethod
    def plot(fig, data):
        x, y, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        x_plot_ratio = 1e-9
        x_plot_label = 'GHz'
        if isinstance(y, np.ndarray):
            a = ax1.imshow(amp, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                        min(y), max(y)),
                           aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)*x_plot_ratio, max(x)*x_plot_ratio, 
                                    min(y), max(y)),
                       aspect='auto', origin='lower', interpolation='nearest')
            fig.colorbar(a)
        else:
            ax1.plot(x*x_plot_ratio, amp)
            ax2.plot(x*x_plot_ratio, pha)
        ax1.set_xlabel('freq (GHz))')
        ax1.set_ylabel('amp (a.u)')
        ax2.set_xlabel('freq (GHz))')
        ax2.set_ylabel('pha (a.u)')

In [46]:
three_dim_S21_with_ATS_and_ATT.save(package='ATS')

## 3D S21 sweep mag bias

### with PG_DC

In [47]:
import numpy as np
import asyncio
import qulab


class three_dim_S21_with_ATS_sweep_mag_bias(qulab.Application):
    '''扫磁场偏置，测量S21'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'bias_info':None,
                        'mw_info':None,
                        'sweep_para': 'meas_freq',
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        bias_info = setting_dict['bias_info'][setting_dict['circuit_list'][0]]
        
        DC = qulab.open_resource(bias_info['inst_source'])
        async for y in self.sweep['bias']:
            DC.setValue('Offset', y, ch=bias_info['ch'])
            await asyncio.sleep(0.01)
            app = qulab.make_app('MW.mw_sweep_one_para',parent=self
                                ).sweep([('para_x', self.sweep['para_x'])]
                                       ).with_settings(setting_dict)
            x, time, amp, pha, sweep_para = await app.done()
            yield x, y, amp, pha
        DC.setValue('Offset', 0, ch=bias_info['ch'])

    async def set_bias(self, y):
        await asyncio.sleep(0.01)
    
    def pre_save(self, x, y, amp, pha):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp, pha

    @staticmethod
    def plot(fig, data):
        x, y, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if isinstance(y, np.ndarray):
            ax1.imshow(amp, extent=(min(x)/1e9, max(x)/1e9, min(y)*1e3, max(y)*1e3),
                       aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)/1e9, max(x)/1e9, min(y)*1e3, max(y)*1e3),
                       aspect='auto', origin='lower', interpolation='nearest')
        else:
            ax1.plot(x/1e9, amp)
            ax2.plot(x/1e9, pha)
        ax1.set_title('Amplitude',fontsize=15)
        ax2.set_title('Phase',fontsize=15)
        ax1.set_ylabel('bias (mV)')
        ax1.set_xlabel('frequency (GHz)')
        ax2.set_ylabel('bias (mV)')
        ax2.set_xlabel('frequency (GHz)')

In [48]:
three_dim_S21_with_ATS_sweep_mag_bias.save(package='ATS')

### with GS200

In [49]:
import numpy as np
import skrf as rf
import asyncio
import qulab

class three_dim_S21_sweep_mag_bias_with_GS200(qulab.Application):
    '''扫磁场偏置，测量S21'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'GS200_source': 'GS200',
                        'Mode': 'current', 
                        'Range': '1', 
                        'Level': 0.001,
                        'ProtectCurr': 1.5, 
                        'ProtectVolt': 5,
                        'Power': -30,
                        'Frequency center': 5.0e9,
                        'Frequency span': 100e6,
                        'Number of points': 101,
                        'Bandwidth': 50,
                        'Output': 'ON',
                        'digital_delay': 10e-9,
                        'phase_compensate': True}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 0)
        # 进行参数配置
        GS200 = qulab.open_resource(setting_dict['GS200_source'])
        for setting in setting_dict:
            GS200.setValue(setting, setting_dict[setting])
        async for y in self.sweep['bias']:
            GS200.setValue('Level', y)
            S21_app = qulab.make_app('PNA.S21', parent=self).with_settings(setting_dict)
            freq, re, im, amp, pha = await S21_app.done()
            yield freq, y, amp, pha

    async def set_bias(self, y):
        await asyncio.sleep(0.1)
    
    def pre_save(self, freq, y, amp, pha):
        if self.data.rows > 1:
            freq = freq[0]
        return freq, y, amp, pha

    @staticmethod
    def plot(fig, data):
        x, y, amp, pha = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if isinstance(y, np.ndarray):
            ax1.imshow(amp, extent=(min(x)/1e9, max(x)/1e9, min(y)*1e3, max(y)*1e3),
                       aspect='auto', origin='lower', interpolation='nearest')
            ax2.imshow(pha, extent=(min(x)/1e9, max(x)/1e9, min(y)*1e3, max(y)*1e3),
                       aspect='auto', origin='lower', interpolation='nearest')
        else:
            ax1.plot(x/1e9, amp)
            ax2.plot(x/1e9, pha)
        ax1.set_title('Amplitude',fontsize=15)
        ax2.set_title('Phase',fontsize=15)
        ax1.set_ylabel('current bias (mA)')
        ax1.set_xlabel('frequency (GHz)')
        ax2.set_ylabel('current bias (mA)')
        ax2.set_xlabel('frequency (GHz)')

In [50]:
three_dim_S21_sweep_mag_bias_with_GS200.save(package='PNA')

## qubit spectroscopy

### spectroscopy line

In [51]:
import numpy as np
import asyncio
import qulab

class spectroscopy_line(qulab.Application):
    '''
    对两台MW进行功率和频率扫描
    可选为meas_mw_freq、meas_mw_power、drive_mw_freq、drive_mw_power
    输入格式为
    app.sweep([('para_x', ***)])
    在with_settings中的sweep_para说明para_x物理上对应上述4组中的哪组
    利用采集卡进行数据采集
    通过该app可以构建S21参数（2D和3D）扫描、qubit能谱测试、单比特时域测试等
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'sweep_para': 'drive_freq',
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # 获取config_info
        mw_info = setting_dict['mw_info']
        if not mw_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            mw_info = config['mw_info']     
        ## 设置MW, 扫描参数进行测量
        if isinstance(setting_dict['sweep_para'],list):
            sweep_para = setting_dict['sweep_para'][0]
        else:
            sweep_para = setting_dict['sweep_para']
        async for x in self.sweep['para_x']:
            for circuit in setting_dict['circuit_list']:
                if 'drive' in sweep_para and 'power' in sweep_para:
                    mw_info['drive_info'][circuit]['power'] = x
                elif 'drive' in sweep_para and 'freq' in sweep_para:
                    mw_info['drive_info'][circuit]['frequency'] = x
                elif 'meas' in sweep_para and 'power' in sweep_para:
                    mw_info['meas_info'][circuit]['power'] = x
                elif 'meas' in sweep_para and 'freq' in sweep_para:
                    mw_info['meas_info'][circuit]['frequency'] = x
            setting_dict[sweep_para] = x
            set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings({'circuit_list': setting_dict['circuit_list'],
                                                                                'mw_info': mw_info})
            await set_mw_app.done()
            
            if 'ATS' in setting_dict['child_app']:
                ## 采集卡进行一次数据采集
                ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                I, Q = await ats_app.done()
                if setting_dict['result_process_app'] is None:
                    print('result process app is required!')
                    break
                else:
                    setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                    process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                                ).with_settings(setting_dict['result_process_app_setting'])
                    result = await process_app.done()
                    yield x, result[0]
            else:
                print('This child app is not supported')
                break
            
    async def set_para_x(self, x):
        await asyncio.sleep(0.02)
                  
    @staticmethod
    def plot(fig, data):
        x, amp = data
        if isinstance(amp[0], np.ndarray):
            amp = np.array(amp).T
            for idx in range(len(amp)):
                row = int((len(amp)+1)/2)
                col = 2
                ax = fig.add_subplot(row,col,idx+1)
                ax.plot(x/1e9,amp[idx],'r-')
                if idx==0:
                    ax.set_ylabel('amp(a.u)')
            ax.set_xlabel('freq(GHz)')
        else:
            ax = fig.add_subplot(111)
            ax.plot(x/1e9,amp[0],'r-')
            ax.set_ylabel('amp(a.u)')
            ax.set_xlabel('freq(GHz)')

In [52]:
spectroscopy_line.save(package='ATS')

### 3D spectroscopy sweep bias

In [53]:
import numpy as np
import asyncio
import qulab

class three_dim_spectroscopy_sweep_bias(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'bias_info': None,
                        'sweep_para': 'drive_freq',
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        # 获取config_info
        mw_info = setting_dict['mw_info']
        bias_info = setting_dict['bias_info']
        if not mw_info:
            meas_config_app = qulab.make_app('base.meas_config',parent=self)
            config = await meas_config_app.done()
            config = config[0]
            mw_info = config['mw_info']
            bias_info = config['bias_info']
            
        ## 设置MW, 扫描参数进行测量
        sweep_para = setting_dict['sweep_para']
        DC = qulab.open_resource(bias_info[setting_dict['circuit_list'][0]]['inst_source'])
        async for y in self.sweep['bias']:
            DC.setValue('Offset', y, ch=bias_info[setting_dict['circuit_list'][0]]['ch'])
            await asyncio.sleep(0.01)
            app = qulab.make_app('ATS.spectroscopy_line',parent=self).sweep([
                ('para_x', self.sweep['para_x'])]).with_settings(setting_dict)
            x, amp = await app.done()
            yield x, y, amp
        DC.setValue('Offset', 
                    bias_info[setting_dict['circuit_list'][0]]['amp'], 
                    ch=bias_info[setting_dict['circuit_list'][0]]['ch'])
        
    async def set_bias(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp
    
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        if isinstance(y, np.ndarray):
            amp = np.array(amp).T
            for idx in range(len(amp)):
                row = int((len(amp)+1)/2)
                col = 2
                ax = fig.add_subplot(row,col,idx+1)
                a = ax.imshow(amp[idx], extent=(min(y)*1000, max(y)*1000, min(x)/1e9, max(x)/1e9),
                              aspect='auto', origin='lower', interpolation='nearest')
                if idx==0:
                    ax.set_ylabel('freq(GHz)')
            ax.set_xlabel('bias(mV)')

In [54]:
three_dim_spectroscopy_sweep_bias.save(package='ATS')

### 3D spectroscopy sweep z control

In [55]:
import numpy as np
import asyncio
import qulab

class three_dim_spectroscopy_sweep_z_control(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None, 
                        'qubit_info': None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        length = len(setting_dict['qubit_info'])
        if idx >= length-1:
            z_gate = 'S'+str(idx)+'_'+str(idx-1)
        else:
            z_gate = 'S'+str(idx)+'_'+str(idx+1)
        setting_dict['gate_list'] = [['X'+str(idx), z_gate],]
        
        #设置AWG为方波，用于能谱测试
        qubit_info[setting_dict['circuit_list'][0]]['pi_array'][0] = 1e-6
        two_bit_gate_info['swap_gate'][z_gate][0] = 1e-6
        async for y in self.sweep['bias']:
            two_bit_gate_info['swap_gate'][z_gate][1] = y
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            await asyncio.sleep(0.5)
            
            app = qulab.make_app('ATS.spectroscopy_line',parent=self).sweep([
                ('para_x', self.sweep['para_x'])]).with_settings(setting_dict)
            x, amp = await app.done()
            yield x, y, amp
        
    async def set_bias(self, y):
        await asyncio.sleep(0.2)
        
    def pre_save(self, x, y, amp):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp
    
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        if isinstance(y, np.ndarray):
            amp = np.array(amp).T
            for idx in range(len(amp)):
                row = int((len(amp)+1)/2)
                col = 2
                ax = fig.add_subplot(row,col,idx+1)
                a = ax.imshow(amp[idx], extent=(min(y), max(y), min(x)/1e9, max(x)/1e9),
                              aspect='auto', origin='lower', interpolation='nearest')
                if idx==0:
                    ax.set_ylabel('freq(GHz)')
            ax.set_xlabel('bias(V)')

In [56]:
three_dim_spectroscopy_sweep_z_control.save(package='ATS')

## optimize readout point

### find readout freq

In [57]:
import numpy as np
import asyncio
import qulab

class find_readout_freq(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'drive_mw_source': None, 
                        'meas_mw_source': None, 
                        'drive_freq': None, 
                        'meas_freq': None,
                        'drive_power': None, 
                        'meas_power': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        mw_drive = qulab.open_resource(setting_dict['drive_mw_source'])
        mw_meas = qulab.open_resource(setting_dict['meas_mw_source'])
        
        mw_drive.setValue('Frequency',setting_dict['drive_freq'])
        mw_drive.setValue('Power',setting_dict['drive_power'])
        mw_meas.setValue('Frequency',setting_dict['meas_freq'])
        mw_meas.setValue('Power',setting_dict['meas_power'])
        mw_meas.setValue('Output','ON')
        
        ## 设置MW
        mw_app = qulab.make_app('MW.mw_control_base').with_settings(
            setting_dict)
        await mw_app.done()
        
        async for x in self.sweep['readout_freq']:
            mw_drive.setValue('Output','OFF')
            mw_meas.setValue('Frequency',x)
            await asyncio.sleep(0.1)
            app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            data = await app.done()
            setting_dict['result_process_app_setting'].update({'I': data[0], 
                                                               'Q': data[1],})
            pro_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                    ).with_settings(setting_dict['result_process_app_setting'])
            base_amp, base_amp_I, base_amp_Q = await pro_app.done()
            
            mw_drive.setValue('Output','ON')
            await asyncio.sleep(0.1)
            app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            data = await app.done()
            setting_dict['result_process_app_setting'].update({'I': data[0], 
                                                               'Q': data[1],})
            pro_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                    ).with_settings(setting_dict['result_process_app_setting'])
            signal_amp, signal_amp_I, signal_amp_Q = await pro_app.done()
            
            yield x, np.sqrt((signal_amp_I-base_amp_I)**2+(signal_amp_Q-base_amp_Q)**2), signal_amp_I-base_amp_I, signal_amp_Q-base_amp_Q

    async def set_readout_freq(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp, amp_I, amp_Q = data
        ax = fig.add_subplot(121)
        ax1 = fig.add_subplot(122)
        ax.plot(x/1e9, amp, 'b-*')
        ax1.plot(x/1e9, amp_I, 'r-*')
        ax1.plot(x/1e9, amp_Q, 'b-*')
        ax.set_xlabel('readout freq (GHz)')
        ax.set_ylabel('amp difference (a.u.)')
        ax1.set_xlabel('readout freq (GHz)')
        ax1.set_ylabel('amp difference (a.u.)')

In [58]:
find_readout_freq.save(package='ATS')

### find readout amp and freq

In [59]:
import numpy as np
import asyncio
import qulab

class find_readout_amp_and_freq(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'qubit_info': None,
                        'sweep_para': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        setting_dict.update({'sweep_para': 'meas_freq',})
        
        #设置AWG
        # 变化meas pulse的幅度，进行读出测试
        drive_power = setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power']
        drive_freq = setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency']
        async for y in self.sweep['readout_amp']:
            # 设置meas awg
            qubit_info[setting_dict['circuit_list'][0]]['probe_amp'] = y
            meas_pulse_app = qulab.make_app('base.meas_pulse_generate',parent=self
                                           ).with_settings({'circuit_list':setting_dict['circuit_list'],
                                                            'meas_info': setting_dict['mw_info']['meas_info'],
                                                            'qubit_info': qubit_info})
            meas_pulse = await meas_pulse_app.done()
            meas_pulse = meas_pulse[0]
            set_meas_awg_app = qulab.make_app('base.map_meas_pulse',parent=self).with_settings({'meas_pulse': meas_pulse})
            state = await set_meas_awg_app.done()
            # 分别测量有驱动和无驱动情况下的结果
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power'] = -50
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = 0.1e9
            S21_app = qulab.make_app('MW.mw_sweep_one_para',parent=self
                                    ).sweep([('para_x', self.sweep['readout_freq'])]
                                           ).with_settings(setting_dict)
            x, time, base_I, base_Q, sweep_para = await S21_app.done()
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = drive_freq
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power'] = drive_power
            S21_app = qulab.make_app('MW.mw_sweep_one_para',parent=self
                                    ).sweep([('para_x', self.sweep['readout_freq'])]
                                           ).with_settings(setting_dict)
            x, time, signal_I, signal_Q, sweep_para = await S21_app.done()
            distance = np.sqrt((base_I-signal_I)**2+(base_Q-signal_Q)**2)
            yield x, y, distance, base_I, base_Q, signal_I, signal_Q

    async def set_readout_amp(self, x):
        await asyncio.sleep(0.02)
    
    def pre_save(self, x, y, distance, base_I, base_Q, signal_I, signal_Q):
        if self.data.rows > 1:
            x = x[0]
        return x, y, distance, base_I, base_Q, signal_I, signal_Q
    
    
    @staticmethod
    def plot(fig, data):
        x, y, distance, base_I, base_Q, signal_I, signal_Q = data
        ax1 = fig.add_subplot(131)
        ax2 = fig.add_subplot(132)
        ax3 = fig.add_subplot(133)
        
        if isinstance(y, np.ndarray):
            a = ax1.imshow(distance, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                                                        aspect='auto', 
                                                        origin='lower', 
                                                        interpolation='nearest')
            ax2.imshow(base_I-signal_I, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                                         aspect='auto',
                                         origin='lower', 
                                         interpolation='nearest')
            ax3.imshow(base_Q-signal_Q, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                                           aspect='auto',
                                           origin='lower', 
                                           interpolation='nearest')
            fig.colorbar(a)
        else:
            ax1.plot(x,distance,'r-*')
            ax2.plot(x,base_I-signal_I,'r-*')
            ax3.plot(x,base_Q-signal_Q,'r-*')
        ax1.set_ylabel('readout_amp')
        ax1.set_xlabel('readout_freq')
        ax2.set_xlabel('readout_freq')
        ax3.set_xlabel('readout_freq')
        ax1.set_title('distance',fontsize=10)
        ax2.set_title('I_diff',fontsize=10)
        ax3.set_title('Q_diff',fontsize=10)

In [60]:
find_readout_amp_and_freq.save(package='ATS')

### find readout amp and freq for single shot

In [61]:
import numpy as np
import asyncio
import qulab
from qulab.tools.wavedata import *

class find_readout_amp_and_freq_for_single_shot(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': None,
                        'mw_info': None,
                        'qubit_info': None,
                        'sweep_para': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': None,
                        'result_process_app_setting': dict()}
        
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        setting_dict.update({'sweep_para': 'meas_freq',})
        
        #设置AWG
        # 变化meas pulse的幅度，进行读出测试
        drive_power = setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power']
        drive_freq = setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency']
        async for y in self.sweep['readout_amp']:
            # 设置meas awg
            qubit_info[setting_dict['circuit_list'][0]]['probe_amp'] = y
            meas_pulse_app = qulab.make_app('base.meas_pulse_generate',parent=self
                                           ).with_settings({'circuit_list':setting_dict['circuit_list'],
                                                            'meas_info': setting_dict['mw_info']['meas_info'],
                                                            'qubit_info': qubit_info})
            meas_pulse = await meas_pulse_app.done()
            meas_pulse = meas_pulse[0]
            set_meas_awg_app = qulab.make_app('base.map_meas_pulse',parent=self).with_settings({'meas_pulse': meas_pulse})
            state = await set_meas_awg_app.done()
            # 分别测量有驱动和无驱动情况下的结果
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power'] = -50
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = 0.1e9
            S21_app = qulab.make_app('MW.mw_sweep_one_para',parent=self
                                    ).sweep([('para_x', self.sweep['readout_freq'])]
                                           ).with_settings(setting_dict)
            x, time, base_I, base_Q, sweep_para = await S21_app.done()
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = drive_freq
            setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['power'] = drive_power
            S21_app = qulab.make_app('MW.mw_sweep_one_para',parent=self
                                    ).sweep([('para_x', self.sweep['readout_freq'])]
                                           ).with_settings(setting_dict)
            x, time, signal_I, signal_Q, sweep_para = await S21_app.done()
            
            # 根据IQ值处理得到每种条件下的信噪比
            snr_list = []
            for idx in range(len(x)):
                distance = np.sqrt((np.mean(base_I[idx])-np.mean(signal_I[idx]))**2
                                   +(np.mean(base_Q[idx])-np.mean(signal_Q[idx]))**2)
                base_I_std = np.std(base_I[idx])
                base_Q_std = np.std(base_Q[idx])
                signal_I_std = np.std(signal_I[idx])
                signal_Q_std = np.std(signal_Q[idx])
                SNR = (np.sqrt(base_I_std**2+base_Q_std**2)+np.sqrt(signal_I_std**2+signal_Q_std**2))/2
                snr_list.append(distance/SNR)
            snr_list = np.array(snr_list)
            yield x, y, snr_list

    async def set_readout_amp(self, x):
        await asyncio.sleep(0.02)
    
    def pre_save(self, x, y, snr_list):
        if self.data.rows > 1:
            x = x[0]
        return x, y, snr_list
    
    
    @staticmethod
    def plot(fig, data):
        x, y, snr_list = data
        ax = fig.add_subplot(111)     
        if isinstance(y, np.ndarray):
            a = ax.imshow(snr_list, extent=(min(x)/1e9, max(x)/1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
            fig.colorbar(a)
        else:
            ax.plot(x,snr_list,'r-*')
        ax.set_ylabel('readout_amp')
        ax.set_xlabel('readout_freq')
        ax.set_title('SNR',fontsize=10)

In [62]:
find_readout_amp_and_freq_for_single_shot.save(package='ATS')

## Rabi

### rabi line sweep drive time

In [63]:
import numpy as np
import asyncio
import qulab

class rabi_line_sweep_drive_time(qulab.Application):
    '''单比特rabi试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 扫描驱动长度，进行rabi测试
        idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(idx),]
        async for x in self.sweep['drive_time']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][0] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_drive_time(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(abs(x)*1e6, amp1, 'b-*')
        ax1.set_xlabel('drive time (us)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(abs(x)*1e6, amp2, 'b-*')
        ax2.set_xlabel('drive time (us)')
        ax2.set_ylabel('amp (a.u.)')

In [64]:
rabi_line_sweep_drive_time.save(package='single_bit')

### rabi line sweep drive amp

In [65]:
import numpy as np
import asyncio
import qulab

class rabi_line_sweep_drive_amp(qulab.Application):
    '''单比特rabi试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 扫描驱动长度，进行rabi测试
        idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(idx),]
        async for x in self.sweep['drive_amp']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][1] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_drive_amp(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(abs(x), amp1, 'b-*')
        ax1.set_xlabel('drive amp (a.u.)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(abs(x), amp2, 'b-*')
        ax2.set_xlabel('drive amp (a.u.)')
        ax2.set_ylabel('amp (a.u.)')

In [66]:
rabi_line_sweep_drive_amp.save(package='single_bit')

## T1

### T1 line

In [67]:
import numpy as np
import asyncio
import qulab

class T1_line(qulab.Application):
    '''单比特T1试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 扫描驱动后等待时间，进行T1测试
        idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(idx),'I'+str(idx)]
        async for x in self.sweep['delay_time']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]

    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)

    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e6, amp1, 'b-*')
        ax1.set_xlabel('time (us)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e6, amp2, 'b-*')
        ax2.set_xlabel('time (us)')
        ax2.set_ylabel('amp (a.u.)')

In [68]:
T1_line.save(package='single_bit')

## Ramsey

### ramsey line

In [69]:
import numpy as np
import asyncio
import qulab

class ramsey_line(qulab.Application):
    '''单比特ramsey试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        '2nd_pulse': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
                
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 扫描等待时间，进行ramsey测试
        async for x in self.sweep['delay_time']:
            idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
            if x > 0:
                # 区分不同的ramsey
                if setting_dict['2nd_pulse']:
                    setting_dict['gate_list'] = ['X2p'+str(idx),'I'+str(idx),setting_dict['2nd_pulse']+str(idx)]
                else:
                    setting_dict['gate_list'] = ['X2p'+str(idx),'I'+str(idx),'X2p'+str(idx)]
            else:
                # 区分不同的ramsey
                if setting_dict['2nd_pulse']:
                    setting_dict['gate_list'] = ['X2p'+str(idx),'X'+str(idx),setting_dict['2nd_pulse']+str(idx)]
                else:
                    setting_dict['gate_list'] = ['X2p'+str(idx),'X'+str(idx),'X2p'+str(idx)] 
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]

    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)

    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e6, amp1, 'b-*')
        ax1.set_xlabel('time (us)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e6, amp2, 'b-*')
        ax2.set_xlabel('time (us)')
        ax2.set_ylabel('amp (a.u.)')

In [70]:
ramsey_line.save(package='single_bit')

### ramsey sweep bias amp

In [71]:
import numpy as np
import asyncio
import qulab

class ramsey_sweep_bias(qulab.Application):
    '''单比特ramsey试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        '2nd_pulse': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
                
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        #生成门序列
        idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        length = len(setting_dict['qubit_info'])
        if idx >= length-1:
            z_gate = 'S'+str(idx)+'_'+str(idx-1)
        else:
            z_gate = 'S'+str(idx)+'_'+str(idx+1)
            
        # 生成bias信号
        setting_dict['two_bit_gate_info']['swap_gate'][z_gate][0] = setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4]
        async for x in self.sweep['bias']:
            if setting_dict['2nd_pulse']:
                setting_dict['gate_list'] = ['X2p'+str(idx),['I'+str(idx),z_gate],setting_dict['2nd_pulse']+str(idx)]
            else:
                setting_dict['gate_list'] = ['X2p'+str(idx),['I'+str(idx),z_gate],'X2p'+str(idx)]
            setting_dict['two_bit_gate_info']['swap_gate'][z_gate][1] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]

    async def set_bias(self, x):
        await asyncio.sleep(0.02)

    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x, amp1, 'b-*')
        ax1.set_xlabel('bias (V)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x, amp2, 'b-*')
        ax2.set_xlabel('time (V)')
        ax2.set_ylabel('amp (a.u.)')

In [72]:
ramsey_sweep_bias.save(package='single_bit')

## Spin echo

### echo line

In [73]:
import numpy as np
import asyncio
import qulab

class echo_line(qulab.Application):
    '''单比特echo试验'''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
                
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        async for x in self.sweep['delay_time']:
            idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
            if x > 0:
                setting_dict['gate_list'] = ['X2p'+str(idx),'I'+str(idx),'X'+str(idx),
                                             'I'+str(idx),'X2p'+str(idx)]
            else:
                setting_dict['gate_list'] = ['X2p'+str(idx),'X'+str(idx),'X2p'+str(idx)] 
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]

    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)

    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e6, amp1, 'b-*')
        ax1.set_xlabel('time (us)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e6, amp2, 'b-*')
        ax2.set_xlabel('time (us)')
        ax2.set_ylabel('amp (a.u.)')

In [74]:
echo_line.save(package='single_bit')

# Clifford RBM

## RBM_single_point

In [75]:
import numpy as np
import asyncio
import qulab
from qulab.tools.cliffordgroup import *

class RBM_single_point(qulab.Application):
    '''
    根据是否给定待标定门的序号，进行参考序列或者标定序列中一个点的测量 
    输入参数，都是采用with_settings方式输入:
    target_gate_num: 0-23之间的数组，对应待标定门，若未None则表示进行参考序列测量
    sequence_num: RBM序列门操作数
    repeat: 重复次数
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'target_gate_num': None,
                        'sequence_num': 1,
                        'rbm_repeat': 1,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        for idx in range(setting_dict['rbm_repeat']):
            ## 生成awg波形,含DRAG修正
            RB_G=cliffordGroup()
            if setting_dict['target_gate_num']:
                RB_seq=cliffordGroup.rbm_seq(RB_G,size=setting_dict['sequence_num'],
                                             group=None,ref=setting_dict['target_gate_num'])
            else:
                RB_seq=cliffordGroup.rbm_seq(RB_G,size=setting_dict['sequence_num'],
                                             group=None,ref=[])
            gate_list = list()
            for gate in RB_seq:
                gate_list.append(gate+str(qubit_idx))
            setting_dict['gate_list'] = gate_list
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield result[0][0]
            else:
                yield result[0]
        
    @staticmethod
    def plot(fig, data):
        ax = fig.add_subplot(111)
        ax.plot(data, 'ro')
        ax.set_xlabel('repeation')
        ax.set_ylabel('sequency fidelity')

In [76]:
RBM_single_point.save(package='RBM')

## RBM_line

In [77]:
import numpy as np
import asyncio
import qulab

class RBM_line(qulab.Application):
    '''
    根据是否给定待标定门的序号，进行参考序列或者标定序列测量
    app.sweep([sequence_num]): 待测量的RBM序列门操作数的array
    
    输入参数，都是采用with_settings方式输入:
    target_gate_num: 0-23之间的数组，对应待标定门，若未None则表示进行参考序列测量
    sequence_num: RBM序列门操作数
    repeat: 重复次数
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'target_gate_num': None,
                        'sequence_num': 1,
                        'rbm_repeat': 1,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        async for x in self.sweep['sequence_num']:
            setting_dict['sequence_num'] = x
            RBM_single_point_app = qulab.make_app('RBM.RBM_single_point',
                                                  parent=self).with_settings(setting_dict)
            fidelity = await RBM_single_point_app.done()
            yield x, np.mean(fidelity)
            
    async def set_sequence_num(self, x):
        await asyncio.sleep(0.02)
        
    @staticmethod
    def plot(fig, data):
        m, F = data
        ax = fig.add_subplot(111)
        ax.plot(m, F, 'r-o')
        ax.set_xlabel('sequence num')
        ax.set_ylabel('sequency fidelity')

In [78]:
RBM_line.save(package='RBM')

## 3D_RBM_sweep_one_parameter

In [79]:
import numpy as np
import asyncio
import qulab

class three_dim_RBM_sweep_one_para(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'RBM_sweep_para': None,
                        'target_gate_num': None,
                        'rbm_repeat': 1,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        drive_info = setting_dict['drive_info']
        
         ## 临时记录需要进行扫描的para
        para = setting_dict['RBM_sweep_para']
        ## 扫描para_y
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        async for y in self.sweep['para_list']:
            if para=='DRAG_ratio':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['DRAG_ratio'] = y
            elif para=='carry_cali_I_amp':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][0][0] = y
            elif para=='carry_cali_I_offset':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][0][1] = y
            elif para=='carry_cali_Q_amp':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][0] = y
            elif para=='carry_cali_Q_offset':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][1] = y
            elif para=='carry_cali_Q_pha':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][2] = y
            elif para=='pi_length':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][0] = y
            elif para=='pi_amp':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][1] = y
            elif para=='pi_2_length':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][2] = y
            elif para=='pi_2_amp':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][3] = y
            else:
                setting_dict[para_y] = y
            
            RBM_line_app = qulab.make_app('RBM.RBM_line',parent=self).with_settings(setting_dict)
            x, fidelity = await RBM_line_app.done()
            yield x, y, fidelity
    
    async def set_para_list(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, fidelity):
        if self.data.rows > 1:
            x = x[0]
        return x, y, fidelity
    
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        ax = fig.add_subplot(111)
        if isinstance(y, np.ndarray):
            if isinstance(x, np.ndarray):
                a = ax.imshow(amp, extent=(min(x), max(x), min(y), max(y)),
                              aspect='auto', origin='lower', interpolation='nearest')
                fig.colorbar(a)
            else:
                ax.plot(y,amp, 'r-o')
                ax.set_xlabel('sweep para')
                ax.set_ylabel('fidelity')
        else:
            ax.plot(x, amp, 'r-o')
        ax.set_xlabel('sequence num')
        ax.set_ylabel('sweep para')

In [80]:
three_dim_RBM_sweep_one_para.save(package='RBM')

## RBM sweep f01 and another para

In [81]:
import numpy as np
import asyncio
import qulab

class RBM_sweep_f01_and_another_para(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'RBM_sweep_two_para': None,
                        'target_gate_num': None,
                        'sequence_num': 1,
                        'repeat_num': 1,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        drive_info = setting_dict['drive_info']
        
         ## 临时记录需要进行扫描的para
        para_x = setting_dict['RBM_sweep_two_para'][0]
        para_y = setting_dict['RBM_sweep_two_para'][1]
        ## 扫描para_y
        async for y in self.sweep['para_y']:
            if para=='DRAG_ratio':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['DRAG_ratio'] = y
            elif para=='carry_cali_I_amp':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][0][0] = y
            elif para=='carry_cali_I_offset':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][0][1] = y
            elif para=='carry_cali_Q_amp':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][0] = y
            elif para=='carry_cali_Q_offset':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][1] = y
            elif para=='carry_cali_Q_pha':
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['IQ_cali_array'][1][2] = y
            elif para=='pi_length':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][0] = y
            elif para=='pi_amp':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][1] = y
            elif para=='pi_2_length':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][2] = y
            elif para=='pi_2_amp':
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][3] = y
            else:
                setting_dict[para_y] = y
            
            drive_mw = qulab.open_resource(setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['inst_source'])
            amp = np.zeros(len(self.sweep['f01']))
            for index in range(setting_dict['repeat_num']):
                RBM_point_app = qulab.make_app('RBM.RBM_single_point',parent=self).with_settings(setting_dict)
                await RBM_point_app.done()
                
                x_x = []
                amp_x = []
                async for x in self.sweep['f01']:
                    x_x.append(x)
                    drive_mw.setValue('Frequency', int(x-float(setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['carry'])))
                    await asyncio.sleep(0.02)
                    ## 采集卡进行一次数据采集
                    ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                    I, Q = await ats_app.done()
            
                    setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                    process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                                ).with_settings(setting_dict['result_process_app_setting'])
                    result = await process_app.done()
                    if isinstance(result[0], np.ndarray):
                        amp_x.append(result[0][0])
                    else:
                        amp_x.append(result[0])
                amp_x=np.array(amp_x)
                amp += amp_x
            yield x_x, y, np.array(amp)/setting_dict['repeat_num']
    
    async def set_f01(self, x):
        await asyncio.sleep(0.02)
    
    async def set_para_y(self, y):
        await asyncio.sleep(0.02)
    
    def pre_save(self, x, y, amp):
        if isinstance(y, np.ndarray):
            x = x[0]
        return x, y, amp 
        
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        ax = fig.add_subplot(111)
        if isinstance(y, np.ndarray):
            a = ax.imshow(amp.T, extent=(min(y), max(y), min(x), max(x)),
                          aspect='auto', origin='lower', interpolation='nearest')
            ax.set_ylabel('para_x')
            ax.set_xlabel('para_y')
            fig.colorbar(a)
        else:
            ax.plot(x,amp,'r-*')
            ax.set_xlabel('para_x')
            ax.set_ylabel('amp')  

In [82]:
RBM_sweep_f01_and_another_para.save(package='RBM')

# XEB

## XEB single point

In [83]:
import numpy as np
import asyncio
import qulab
from qulab.tools.wavedata import *
from qulab.tools.XEBgroup import *

class XEB_single_point(qulab.Application):
    '''
    根据是否给定待标定门的序号，进行参考序列或者标定序列中一个点的测量 
    输入参数，都是采用with_settings方式输入:
    target_gate_num: 25-26之间的数组，对应待标定门，若未None则表示进行参考序列测量
    sequence_num: RBM序列门操作数
    repeat: 重复次数
    carry: sideband频率
    pi_array: 格式为[100e-9,1,50e-9,1]，表示pi、pi_2的长度、幅度
    wave_length: 波形总长度
    carry_cali: IQ mixer校正矩阵
    awg_source: awg设备name
    drive_mw_source: 驱动mw设备name
    meas_mw_source: 测量mw设备name
    f01: qubit频率
    drive_power: 驱动mw功率
    meas_power: 测量mw功率
    meas_freq: 测量mw频率
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'target_gate_num': None,
                        'sequence_num': 1,
                        'rbm_repeat': 1,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
                
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        for repeat in range(setting_dict['rbm_repeat']):
            ## 生成序列
            xeb_G=xebgroup()
            seq1, seq2, seq, prob_array=xebgroup.xeb_seq(xeb_G,size=setting_dict['sequence_num'],
                                                         group=None,
                                                         target=setting_dict['target_gate_num'])
            gate_list = list()
            seq_idx = 1
            for seq_item in seq:
                ## Q1的xy门操作
                if seq_idx%3==1:
                    gate_list.append(seq_item+str(qubit_idx_0))
                ## Q2的xy门操作
                elif seq_idx%3==2:
                    gate_list.append(seq_item+str(qubit_idx_1))
                ## z门操作
                else:
                    if setting_dict['target_gate_num'] == 25:
                        gate_list.append('CP'+str(qubit_idx_0)+'_'+str(qubit_idx_1))
                    else:
                        gate_list.append('iS'+str(qubit_idx_0)+'_'+str(qubit_idx_1))
                seq_idx += 1
            ## 生成awg波形,含DRAG修正
            setting_dict['gate_list'] = gate_list
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集,需设置为单发测量，得到各状态概率
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            IQ,prob,center = await process_app.done()
            
            S_ie = -np.sum(np.log(prob_array))/4
            S_me = -np.sum(prob*np.log(prob_array))
            S_ee = -np.sum(prob_array*np.log(prob_array))
            yield (S_ie-S_me)/(S_ie-S_ee)
        
    @staticmethod
    def plot(fig, data):
        ax = fig.add_subplot(111)
        ax.plot(data, 'ro')
        ax.set_xlabel('repeation')
        ax.set_ylabel('xeb fidelity')

In [100]:
"""
from pathlib import Path
from ruamel import yaml

config_path = '/Users/liuqichun/Quantum_Lab/qulab/tools/auto_meas_app/config_info.yaml'
path = Path(config_path)
config = yaml.safe_load(path.read_text())


setting_dict = {'target_gate_num': 25,
                'sequence_num': 100,
                'rbm_repeat': 5,
                'circuit_list': ['Qubit0',],
                'mw_info':None,
                'qubit_info':None,
                'two_bit_gate_info': None, 
                'child_app': 'ATS_PACK.ats_acquire',
                'child_app_setting': dict(),
                'result_process_app': 'ATS_PACK.ats_result_process',
                'result_process_app_setting': dict()}
app = qulab.make_app('XEB.XEB_single_point').with_settings(setting_dict)
qulab.make_figure_for_app(app)
app.run()
"""

"\nfrom pathlib import Path\nfrom ruamel import yaml\n\nconfig_path = '/Users/liuqichun/Quantum_Lab/qulab/tools/auto_meas_app/config_info.yaml'\npath = Path(config_path)\nconfig = yaml.safe_load(path.read_text())\n\n\nsetting_dict = {'target_gate_num': 25,\n                'sequence_num': 100,\n                'rbm_repeat': 5,\n                'circuit_list': ['Qubit0',],\n                'mw_info':None,\n                'qubit_info':None,\n                'two_bit_gate_info': None, \n                'child_app': 'ATS_PACK.ats_acquire',\n                'child_app_setting': dict(),\n                'result_process_app': 'ATS_PACK.ats_result_process',\n                'result_process_app_setting': dict()}\napp = qulab.make_app('XEB.XEB_single_point').with_settings(setting_dict)\nqulab.make_figure_for_app(app)\napp.run()\n"

# single bit QST

## QST_single_point

In [101]:
import numpy as np
import asyncio
import qulab

class QST_single_point(qulab.Application):
    '''
    根据输入的QST_index: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定需要进行哪个轴的测量
    根据输入的pre_drive: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），
                    或者waveform，确定在QST之前需要进行哪些操作，若为None则表示无操作
    返回结果为numpy数组，按顺序对应为QST_index中各轴的测量结果（采集卡输出数值）
    # # #
    输入参数，都是采用with_settings方式输入:
    QST_index: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定QST测量轴
    pre_drive: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），
                确定在QST之前需要进行哪些操作，若为None则表示无操作
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'QST_index': ['I',],
                        'pre_drive': None,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        for idx in setting_dict['QST_index']:
            ## 生成awg波形,含DRAG修正
            gate_list = np.append(setting_dict['pre_drive'],idx)
            setting_dict['gate_list'] = gate_list
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield idx, result[0][0]
            else:
                yield idx, result[0]
            
    @staticmethod
    def plot(fig, data):
        idx, amp = data
        ax = fig.add_subplot(111)
        ax.plot(amp, 'ro')
        ax.set_xlabel('QST_index')
        ax.set_ylabel('amp')

In [102]:
QST_single_point.save(package='QST')

## QST_for_rabi_line_sweep_drive_time

In [103]:
import numpy as np
import asyncio
import qulab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

class QST_for_rabi_line_sweep_drive_time(qulab.Application):
    '''
    根据输入的QST_index: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定需要进行哪个轴的测量
    根据输入的pre_drive: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），
                    或者waveform，确定在QST之前需要进行哪些操作，若为None则表示无操作
    返回结果为numpy数组，按顺序对应为QST_index中各轴的测量结果（采集卡输出数值）
    # # #
    输入参数，都是采用with_settings方式输入:
    QST_index: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定QST测量轴
    pre_drive: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n）或者Wavedata格式，
                确定在QST之前需要进行哪些操作，若为None则表示无操作
                Wavedata格式不需要包含DRAG修正，在此会进行DRAG修正
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'QST_index': ['I',],
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 扫描驱动长度，进行rabi测试，只生成驱动pulse，且DRAG和marker都由调用的QST_single_point子app生成
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        async for y in self.sweep['drive_time']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][0] = y
            if y > 0:
                setting_dict['pre_drive'] = ['X'+str(qubit_idx),]
            else:
                setting_dict['pre_drive'] = ['I'+str(qubit_idx),]
            
            QST_app = qulab.make_app('QST.QST_single_point',parent=self).with_settings(setting_dict)
            idx, amp = await QST_app.done()
            yield idx, y, amp
        
    async def set_drive_time(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp   
            
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        amp = np.array(amp)*2-1
        
        s_u = np.linspace(0, 2 * np.pi, 100)
        s_v = np.linspace(0, np.pi, 100)
        s_x = np.outer(np.cos(s_u), np.sin(s_v))
        s_y = np.outer(np.sin(s_u), np.sin(s_v))
        s_z = np.outer(np.ones(np.size(s_u)), np.cos(s_v))
        ax_x = np.sin(s_u)
        ax_y = s_u*0
        ax_z = np.cos(s_u)
        axis = np.linspace(-1.0, 1.0, 10)
        other = np.zeros_like(axis)
        
        grid = plt.GridSpec(10, 1, wspace=0.5, hspace=0.5)
        ax = fig.add_subplot(grid[2:10,0],projection='3d')
        ax1 = fig.add_subplot(grid[0:3,0])
        
        ax.view_init(elev=20,azim=30)
        ax.plot_surface(s_x, s_y, s_z, color='k',alpha=0.05)
        ax.plot(axis,other,other, 'k-', linewidth=1)
        ax.plot(other,axis,other, 'k-', linewidth=1)
        ax.plot(other,other,axis, 'k-', linewidth=1)
        #ax.plot(ax_x,ax_y,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_y,ax_x,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_x,ax_z,ax_y, 'k-', linewidth=0.2)
        ax.text(1.1,0,0,'|x>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,1.1,0,'|y>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,1.1,'|0>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,-1.1,'|1>',verticalalignment="center",horizontalalignment="center")
        if isinstance(y, np.ndarray):
            ax1.plot(y,amp[:,0],'r-*')
            ax1.plot(y,amp[:,1],'g-.')
            ax1.plot(y,amp[:,2],'b-+')
            ax1.set_title('X/Y/Z amp',fontsize=10)
            temp = np.sqrt(amp[:,0]**2+amp[:,1]**2+amp[:,2]**2)
            for idx in range(len(temp)):
                amp[idx] = amp[idx]/temp[idx]
            ax.plot(amp[:,0],amp[:,1],amp[:,2], 'r-*')
        else:
            ax.plot([amp[0],],[amp[1],],[amp[2],], 'r*')

In [104]:
QST_for_rabi_line_sweep_drive_time.save(package='QST')

## QST_for_rabi_line_sweep_drive_amp

In [105]:
import numpy as np
import asyncio
import qulab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

class QST_for_rabi_line_sweep_drive_amp(qulab.Application):
    '''
    根据输入的QST_index: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定需要进行哪个轴的测量
    根据输入的pre_drive: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），
                    或者waveform，确定在QST之前需要进行哪些操作，若为None则表示无操作
    返回结果为numpy数组，按顺序对应为QST_index中各轴的测量结果（采集卡输出数值）
    # # #
    输入参数，都是采用with_settings方式输入:
    QST_index: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定QST测量轴
    pre_drive: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n）或者Wavedata格式，
                确定在QST之前需要进行哪些操作，若为None则表示无操作
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'QST_index': ['I',],
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
                
        ## 扫描驱动长度，进行rabi测试，只生成驱动pulse，且DRAG和marker都由调用的QST_single_point子app生成
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['pre_drive'] = ['X'+str(qubit_idx),]
        async for y in self.sweep['drive_amp']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][1] = y
            QST_app = qulab.make_app('QST.QST_single_point',parent=self).with_settings(setting_dict)
            idx, amp = await QST_app.done()
            yield idx, y, amp
        
    async def set_drive_amp(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp   
            
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        amp = np.array(amp)*2-1
        
        s_u = np.linspace(0, 2 * np.pi, 100)
        s_v = np.linspace(0, np.pi, 100)
        s_x = np.outer(np.cos(s_u), np.sin(s_v))
        s_y = np.outer(np.sin(s_u), np.sin(s_v))
        s_z = np.outer(np.ones(np.size(s_u)), np.cos(s_v))
        ax_x = np.sin(s_u)
        ax_y = s_u*0
        ax_z = np.cos(s_u)
        axis = np.linspace(-1.0, 1.0, 10)
        other = np.zeros_like(axis)
        
        grid = plt.GridSpec(10, 1, wspace=0.5, hspace=0.5)
        ax = fig.add_subplot(grid[2:10,0],projection='3d')
        ax1 = fig.add_subplot(grid[0:3,0])
        
        ax.view_init(elev=20,azim=30)
        ax.plot_surface(s_x, s_y, s_z, color='k',alpha=0.05)
        ax.plot(axis,other,other, 'k-', linewidth=1)
        ax.plot(other,axis,other, 'k-', linewidth=1)
        ax.plot(other,other,axis, 'k-', linewidth=1)
        #ax.plot(ax_x,ax_y,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_y,ax_x,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_x,ax_z,ax_y, 'k-', linewidth=0.2)
        ax.text(1.1,0,0,'|x>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,1.1,0,'|y>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,1.1,'|0>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,-1.1,'|1>',verticalalignment="center",horizontalalignment="center")
        if isinstance(y, np.ndarray):
            ax1.plot(abs(y),amp[:,0],'r-*')
            ax1.plot(abs(y),amp[:,1],'g-.')
            ax1.plot(abs(y),amp[:,2],'b-+')
            ax1.set_title('X/Y/Z amp',fontsize=10)
            temp = np.sqrt(amp[:,0]**2+amp[:,1]**2+amp[:,2]**2)
            #for idx in range(len(temp)):
            #    amp[idx] = amp[idx]/temp[idx]
            ax1.plot(abs(y),1-temp,'k-<')
            ax.plot(amp[:,0],amp[:,1],amp[:,2], 'r-*')
        else:
            ax.plot([amp[0],],[amp[1],],[amp[2],], 'r*')

In [106]:
QST_for_rabi_line_sweep_drive_amp.save(package='QST')

## QST_for_T1

In [107]:
import numpy as np
import asyncio
import qulab
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

class QST_for_T1(qulab.Application):
    '''
    根据输入的QST_index: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定需要进行哪个轴的测量
    根据输入的pre_drive: ['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），
                    或者waveform，确定在QST之前需要进行哪些操作，若为None则表示无操作
    返回结果为numpy数组，按顺序对应为QST_index中各轴的测量结果（采集卡输出数值）
    # # #
    输入参数，都是采用with_settings方式输入:
    QST_index: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n），确定QST测量轴
    pre_drive: 格式为['','',...]（可选为 I/X/Y/X2p/X2n/Y2p/Y2n）或者Wavedata格式，
                确定在QST之前需要进行哪些操作，若为None则表示无操作
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'QST_index': ['I',],
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 只生成驱动pulse，且DRAG和marker都由调用的QST_single_point子app生成
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        async for y in self.sweep['delay_time']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = y
            if y > 0:
                setting_dict['pre_drive'] = ['X'+str(qubit_idx),'I'+str(qubit_idx)]
            else:
                setting_dict['pre_drive'] = ['X'+str(qubit_idx),]
            QST_app = qulab.make_app('QST.QST_single_point',parent=self).with_settings(setting_dict)
            idx, amp = await QST_app.done()
            yield idx, y, amp
        
    async def set_delay_time(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp   
            
    @staticmethod
    def plot(fig, data):
        x, y, amp = data
        amp = np.array(amp)*2-1
        
        s_u = np.linspace(0, 2 * np.pi, 100)
        s_v = np.linspace(0, np.pi, 100)
        s_x = np.outer(np.cos(s_u), np.sin(s_v))
        s_y = np.outer(np.sin(s_u), np.sin(s_v))
        s_z = np.outer(np.ones(np.size(s_u)), np.cos(s_v))
        ax_x = np.sin(s_u)
        ax_y = s_u*0
        ax_z = np.cos(s_u)
        axis = np.linspace(-1.0, 1.0, 10)
        other = np.zeros_like(axis)
        
        grid = plt.GridSpec(10, 1, wspace=0.5, hspace=0.5)
        ax = fig.add_subplot(grid[2:10,0],projection='3d')
        ax1 = fig.add_subplot(grid[0:3,0])
        
        ax.view_init(elev=20,azim=30)
        ax.plot_surface(s_x, s_y, s_z, color='k',alpha=0.05)
        ax.plot(axis,other,other, 'k-', linewidth=1)
        ax.plot(other,axis,other, 'k-', linewidth=1)
        ax.plot(other,other,axis, 'k-', linewidth=1)
        #ax.plot(ax_x,ax_y,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_y,ax_x,ax_z, 'k-', linewidth=0.2)
        ax.plot(ax_x,ax_z,ax_y, 'k-', linewidth=0.2)
        ax.text(1.1,0,0,'|x>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,1.1,0,'|y>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,1.1,'|0>',verticalalignment="center",horizontalalignment="center")
        ax.text(0,0,-1.1,'|1>',verticalalignment="center",horizontalalignment="center")
        if isinstance(y, np.ndarray):
            ax1.plot(abs(y)*1e6,amp[:,0],'r-*')
            ax1.plot(abs(y)*1e6,amp[:,1],'g-.')
            ax1.plot(abs(y)*1e6,amp[:,2],'b-+')
            ax1.set_title('X/Y/Z amp',fontsize=10)
            temp = np.sqrt(amp[:,0]**2+amp[:,1]**2+amp[:,2]**2)
            #for idx in range(len(temp)):
            #    amp[idx] = amp[idx]/temp[idx]
            ax1.plot(abs(y)*1e6,1-temp,'k-<')
            ax.plot(amp[:,0],amp[:,1],amp[:,2], 'r-*')
        else:
            ax.plot([amp[0],],[amp[1],],[amp[2],], 'r*')

In [108]:
QST_for_T1.save(package='QST')

# Two qubits operation

## switch marker calibration

In [109]:
import numpy as np
import asyncio
import qulab

class switch_marker_calibration(qulab.Application):
    '''
    标定XY-Z的线路延时差别
    XY线路加pi脉冲，Z线路加bias脉冲
    标定XY脉冲相对Z脉冲的延时，延时为正值表示滞后
    AWG通道1/2用于XY控制，通道3/4用于Z控制
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'awg_info': None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 只生成驱动pulse，且DRAG和marker都由调用的QST_single_point子app生成
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(qubit_idx),]
        setting_dict['drive_info'] = setting_dict['awg_info']['drive_info']
        async for x in self.sweep['delay_time']:
            setting_dict['drive_info'][setting_dict['circuit_list'][0]]['mk_delay'] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse,
                                                                                                  'drive_info':drive_info})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[0]
            
    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e9, amp1, 'b-*')
        ax1.set_xlabel('delay time (ns)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e9, amp2, 'b-*')
        ax2.set_xlabel('delay time (ns)')
        ax2.set_ylabel('amp (a.u.)')

In [110]:
switch_marker_calibration.save(package='two_bits')

## XY-Z timing calibration

In [111]:
import numpy as np
import asyncio
import qulab

class XY_Z_timing_calibration(qulab.Application):
    '''
    标定XY-Z的线路延时差别
    XY线路加pi脉冲，Z线路加bias脉冲
    标定Z脉冲相对XY脉冲的延时，延时为正值表示滞后
    AWG通道1/2用于XY控制，通道3/4用于Z控制
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'awg_info': None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(qubit_idx),]
        drive_info = setting_dict['awg_info']['drive_info']
        async for x in self.sweep['delay_time']:
            drive_info[setting_dict['circuit_list'][0]]['Z_delay'] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse,
                                                                                                  'drive_info':drive_info})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e9, amp1, 'b-*')
        ax1.set_xlabel('delay time (ns)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e9, amp2, 'b-*')
        ax2.set_xlabel('delay time (ns)')
        ax2.set_ylabel('amp (a.u.)')

In [112]:
XY_Z_timing_calibration.save(package='two_bits')

## Z cross-talk calibtarion

In [113]:
import numpy as np
import asyncio
import qulab

class Z_crosstalk_calibration(qulab.Application):
    '''
    标定两个qubit的Z控制crosstalk
    方式为给定master qubit的Z bias，然后扫描slave qubit的Z bias
    对slave qubit进行固定delay time的Ramsey测试（X2p,Y2p脉冲），得到slave qubit状态x轴投影随bias的变化
    先给出一组两个qubit都不加bias的结果(固定为3个点)，再给出一组给bias的结果，进行对比
    AWG通道1/2用于slave qubit XY控制，通道3用于master qubit的Z控制，通道4用于slave qubit的Z控制
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'ramsey_delay_time': None,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        setting_dict['gate_list'] = ['X2p'+str(qubit_idx_1),
                                     'I'+str(qubit_idx_1),
                                     'X2p'+str(qubit_idx_1)]
        
        if not setting_dict['ramsey_delay_time']:
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = setting_dict['ramsey_delay_time']
        drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
        drive_pulse = await drive_pulse_app.done()
        drive_pulse = drive_pulse[0]
        set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
        state = await set_drive_awg_app.done()
        
        for x in range(3):
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
                
        ## 扫描slave qubit的bias
        async for x in self.sweep['bias_amp']:
            setting_dict['gate_list'] = ['X2p'+str(qubit_idx_1),
                                         ['I'+str(qubit_idx_1),
                                          'S'+str(qubit_idx_0)+'_'+str(qubit_idx_1),
                                          'S'+str(qubit_idx_1)+'_'+str(qubit_idx_0)],
                                         'X2p'+str(qubit_idx_1)]
            setting_dict['two_bit_gate_info']['swap_gate']['S'+str(qubit_idx_1)+'_'+str(qubit_idx_0)][1] = x
            if not setting_dict['ramsey_delay_time']:
                setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = setting_dict['ramsey_delay_time']
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_bias_amp(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        if isinstance(x, np.ndarray):
            if len(x)>3:
                ax1.plot(x[3:], amp1[3:], 'b-*')
                ax2.plot(x[3:], amp2[3:], 'b-*')
                ax1.plot([x[3],x[int((len(x)+3)/2)],x[-1]],amp1[:3],'k->')
                ax2.plot([x[3],x[int((len(x)+3)/2)],x[-1]],amp2[:3],'k->')
            else:
                ax1.plot(x, amp1, 'k->')
                ax2.plot(x, amp2, 'k->')
            ax1.set_xlabel('bias amp (a.u)')
            ax1.set_ylabel('amp (a.u.)')
            ax2.set_xlabel('bias amp (a.u)')
            ax2.set_ylabel('amp (a.u.)')

In [114]:
Z_crosstalk_calibration.save(package='two_bits')

## state swap line sweep bias

In [115]:
import numpy as np
import asyncio
import qulab

class state_swap_sweep_bias(qulab.Application):
    '''
    扫描bias进行 two qubit state swap
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(qubit_idx_0),'S'+str(qubit_idx_0)]
        async for x in self.sweep['bias_amp']:
            setting_dict['two_bit_gate_info']['swap_gate']['S'+str(qubit_idx_0)+'_'+str(qubit_idx_1)][1] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()

            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_bias_amp(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x, amp1, 'b-*')
        ax1.set_xlabel('bias(V)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x, amp2, 'b-*')
        ax2.set_xlabel('bias(V)')
        ax2.set_ylabel('amp (a.u.)')

In [116]:
state_swap_sweep_bias.save(package='two_bits')

## state swap line sweep time

In [117]:
import numpy as np
import asyncio
import qulab

class state_swap_sweep_time(qulab.Application):
    '''
    扫描bias进行 two qubit state swap
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None, 
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        setting_dict['gate_list'] = ['X'+str(qubit_idx_0),'S'+str(qubit_idx_0)] 
        async for x in self.sweep['swap_time']:
            setting_dict['two_bit_gate_info']['swap_gate']['S'+str(qubit_idx_0)+'_'+str(qubit_idx_1)][0] = x
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()

            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            if isinstance(result[0], np.ndarray):
                yield x, result[0][0], result[0][1]
            else:
                yield x, result[0], result[1]
            
    async def set_swap_time(self, x):
        await asyncio.sleep(0.02)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(x*1e9, amp1, 'b-*')
        ax1.set_xlabel('time(ns)')
        ax1.set_ylabel('amp (a.u.)')
        ax2.plot(x*1e9, amp2, 'b-*')
        ax2.set_xlabel('time(ns)')
        ax2.set_ylabel('amp (a.u.)')

In [118]:
state_swap_sweep_time.save(package='two_bits')

## 3D state swap sweep bias and time

In [85]:
import numpy as np
import asyncio
import qulab

class three_dim_state_swap_sweep_bias_and_time(qulab.Application):
    '''
    扫描bias和time进行 two qubit state swap
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        async for y in self.sweep['swap_bias']:
            setting_dict['two_bit_gate_info']['swap_gate']['S'+str(qubit_idx_0)+'_'+str(qubit_idx_1)][1] = y
            app = qulab.make_app('two_bits.state_swap_sweep_time',parent=self
                                ).sweep([('swap_time',self.sweep['swap_time'])]
                                       ).with_settings(setting_dict)
            x, amp1, amp2 = await app.done()
            yield x, y, amp1, amp2
            
    async def set_swap_bias(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp1, amp2):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp1, amp2
            
    @staticmethod
    def plot(fig, data):
        x, y, amp1, amp2 = data
        ax1 = fig.add_subplot(211)
        ax2 = fig.add_subplot(212)
        if isinstance(y, np.ndarray):
            a = ax1.imshow(amp1, extent=(min(x)*1e9, max(x)*1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
            fig.colorbar(a)
            b = ax2.imshow(amp2, extent=(min(x)*1e9, max(x)*1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
            fig.colorbar(b)
        else:
            ax1.plot(x,amp1,'r-*')
            ax2.plot(x,amp2,'r-*')
        ax1.set_ylabel('bias(V)')
        ax2.set_ylabel('bias(V)')
        ax2.set_xlabel('time(ns)')
        ax1.set_title('source qubit',fontsize=10)
        ax2.set_title('target qubit',fontsize=10)

In [86]:
three_dim_state_swap_sweep_bias_and_time.save(package='two_bits')

## cPhase calibration line

In [87]:
import numpy as np
import asyncio
import qulab

class cPhase_cali_line_sweep_time(qulab.Application):
    '''
    默认状态为|Q1,Q2>，扫描Q2的偏置，对Q2进行类ramsey测试
    得到Q1处于基态和激发态两种情况下Q2相位随偏置的演化
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        async for x in self.sweep['cphase_time']:
            temp_out = []
            for Q1_amp in range(2):
                if Q1_amp == 0:
                    setting_dict['gate_list'] = ['I'+str(qubit_idx_0),
                                                 'X2p'++str(qubit_idx_1),
                                                 'CP'+str(qubit_idx_1)+str(qubit_idx_0),
                                                 'X2p'++str(qubit_idx_1)]
                else:
                    setting_dict['gate_list'] = ['X'+str(qubit_idx_0),
                                                 'X2p'++str(qubit_idx_1),
                                                 'CP'+str(qubit_idx_1)+str(qubit_idx_0),
                                                 'X2p'++str(qubit_idx_1)]
                setting_dict['two_bit_gate_info']['cphase_gate']['CP'+str(qubit_idx_1)+str(qubit_idx_0)][0] = x
                drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
                drive_pulse = await drive_pulse_app.done()
                drive_pulse = drive_pulse[0]
                set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
                state = await set_drive_awg_app.done()

                ## 采集卡进行一次数据采集
                ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                I, Q = await ats_app.done()

                setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                            ).with_settings(setting_dict['result_process_app_setting'])
                result = await process_app.done()
                if isinstance(result[0], np.ndarray):
                    temp_out.append(result[0][1])
                else:
                    temp_out.append(result[1])
            yield x, temp_out[0], temp_out[1]
            
    async def set_cphase_time(self, x):
        await asyncio.sleep(0.2)
    
    @staticmethod
    def plot(fig, data):
        x, amp1, amp2 = data
        ax = fig.add_subplot(111)
        ax.plot(x*1e9, amp1, 'b-*')
        ax.plot(x*1e9, amp2, 'r-*')
        ax.set_xlabel('time(ns)')
        ax.set_ylabel('amp (a.u.)')

In [88]:
cPhase_cali_line_sweep_time.save(package='two_bits')

## 3D cPhase calibration

In [89]:
import numpy as np
import asyncio
import qulab

class three_dim_cphase_cali_sweep_bias_and_time(qulab.Application):
    '''
    扫描bias和time进行 cPhase calibration
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        async for y in self.sweep['cphase_bias']:
            setting_dict['two_bit_gate_info']['cphase_gate']['CP'+str(qubit_idx_1)+str(qubit_idx_0)][1] = y
            app = qulab.make_app('two_bits.cPhase_cali_line_sweep_time',parent=self
                                ).sweep([('cphase_time',self.sweep['cphase_time'])]
                                       ).with_settings(setting_dict)
            x, amp1, amp2 = await app.done()
            yield x, y, amp1, amp2
            
    async def set_cphase_bias(self, y):
        await asyncio.sleep(0.02)
        
    def pre_save(self, x, y, amp1, amp2):
        if self.data.rows > 1:
            x = x[0]
        return x, y, amp1, amp2
            
    @staticmethod
    def plot(fig, data):
        x, y, amp1, amp2 = data
        ax1 = fig.add_subplot(311)
        ax2 = fig.add_subplot(312)
        ax3 = fig.add_subplot(313)
        if isinstance(y, np.ndarray):
            a = ax1.imshow(amp2-amp1, extent=(min(x)*1e9, max(x)*1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
            b = ax2.imshow(amp1, extent=(min(x)*1e9, max(x)*1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
            c = ax3.imshow(amp2, extent=(min(x)*1e9, max(x)*1e9, min(y), max(y)),
                           aspect='auto', 
                           origin='lower', 
                           interpolation='nearest')
        else:
            ax1.plot(x,amp2-amp1,'r-*')
            ax2.plot(x,amp1,'r-*')
            ax3.plot(x,amp2,'r-*')
        ax1.set_ylabel('cphase_bias(V)')
        ax2.set_ylabel('cphase_bias(V)')
        ax3.set_ylabel('cphase_bias(V)')
        ax3.set_xlabel('cphase_time(ns)')

In [90]:
three_dim_cphase_cali_sweep_bias_and_time.save(package='two_bits')

## two qubit state prepare

In [91]:
import numpy as np
import asyncio
import qulab

class two_qubit_state_prepare(qulab.Application):
    '''
    用swap方式进行两比特状态制备
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'gate_list': None,
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
       ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        ## 设置awg
        drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
        drive_pulse = await drive_pulse_app.done()
        drive_pulse = drive_pulse[0]
        set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
        state = await set_drive_awg_app.done()

        ## 采集卡进行一次数据采集
        ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
        I, Q = await ats_app.done()
        setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
        process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                    ).with_settings(setting_dict['result_process_app_setting'])
        result = await process_app.done()
        if isinstance(result[0], np.ndarray):
            yield [result[0][0],result[1][0],result[2][0]], [result[0][1],result[1][1],result[2][1]]
        else:
            yield [result[0],result[0]], [result[1],result[1]]
    
    @staticmethod
    def plot(fig, data):
        amp1, amp2 = data
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        ax1.plot(amp1,'r*')
        ax2.plot(amp2,'r*')
        ax1.set_ylabel('amp(a.u)')
        ax1.set_xlabel('amp/I/Q')
        ax2.set_xlabel('amp/I/Q')

In [92]:
two_qubit_state_prepare.save(package='two_bits')

## two_qubit_state_prepare_and_QST

In [93]:
import numpy as np
import asyncio
import qulab
import matplotlib.pyplot as plt

class two_qubit_state_prepare_and_qst(qulab.Application):
    '''
    对两比特进行QST
    source qubit对应为 q0
    target qubit对应为 q1
    两比特态定义为 |q1,q0>
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'pre_drive': None,
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        # 定义two qubit QST序列
        two_qubit_QST_index = [['I','I'],['X','I'],['I','X'],['X2p','I'],
                               ['X2p','X2p'],['X2p','Y2p'],['X2p','X'],['Y2p','I'],
                               ['Y2p','X2p'],['Y2p','Y2p'],['Y2p','X'],['I','X2p'],
                               ['X','X2p'],['I','Y2p'],['X','Y2p'],]
        qubit_idx_0 = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        qubit_idx_1 = int(setting_dict['circuit_list'][1].split('Qubit')[1])
        for QST_idx in two_qubit_QST_index:
            setting_dict['gate_list'] = list(setting_dict['pre_drive'])
            setting_dict['gate_list'].append(QST_idx[0]+str(qubit_idx_0))
            setting_dict['gate_list'].append(QST_idx[1]+str(qubit_idx_1))
            ## 设置awg
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()
            
            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            yield QST_idx, result[1][0], result[1][1], result[1][2], result[1][3]
    
    @staticmethod
    def plot(fig, data):
        idx, prob_00, prob_01, prob_10, prob_11 = data
        two_qubit_QST_index = [['I','I'],['X','I'],['I','X'],['X2p','I'],
                               ['X2p','X2p'],['X2p','Y2p'],['X2p','X'],['Y2p','I'],
                               ['Y2p','X2p'],['Y2p','Y2p'],['Y2p','X'],['I','X2p'],
                               ['X','X2p'],['I','Y2p'],['X','Y2p'],]
        grid = plt.GridSpec(5, 1, wspace=0.5, hspace=0.5)
        ax = fig.add_subplot(grid[0:4,0])
        ax.plot(prob_00, 'k-*')
        ax.plot(prob_01, 'r-*')
        ax.plot(prob_10, 'g-*')
        ax.plot(prob_11, 'b-*')
        plt.xticks(range(len(amp)), two_qubit_QST_index[:len(amp)], rotation=90)
        ax.tick_params(labelsize=8)
        ax.set_xlabel('QST_index')
        ax.set_ylabel('probability')

In [94]:
two_qubit_state_prepare_and_qst.save(package='two_bits')

# N qubits operation

## N qubit state prepare

In [134]:
import numpy as np
import asyncio
import qulab
from qulab.tools.gate_sequence_simulator._GSS import *

class N_qubit_state_prepare(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'gate_list': None,
                        'show_theory_result': False,
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
       ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        ## 设置awg
        drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
        drive_pulse = await drive_pulse_app.done()
        drive_pulse = drive_pulse[0]
        set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
        state = await set_drive_awg_app.done()

        ## 采集卡进行一次数据采集
        ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
        I, Q = await ats_app.done()
        setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
        process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                    ).with_settings(setting_dict['result_process_app_setting'])
        result = await process_app.done()
        
        # 获取理论结果,同时对应测试结果为单发测量结果
        if setting_dict['show_theory_result']:
            simulator = GSS()
            simulator.gate_list = setting_dict['gate_list']
            theory_result = simulator.get_final_probability()
            yield [result[1], theory_result]
        else:
            yield result[0]
    
    @staticmethod
    def plot(fig, data):
        data = np.array(data)
        ax = fig.add_subplot(111)
        if isinstance(data,np.ndarray):
            ax.plot(data[0],'r*',markersize ='10')
            ax.bar(range(data[1].size),data[1],color='lightskyblue')
        else:
            ax.plot(data,'r*',markersize ='10')
        ax.set_ylabel('amp/probability(a.u)')
        ax.set_xlabel('qubit_idx/engien_state_idx')

In [135]:
N_qubit_state_prepare.save(package='N_bits')

## N qubit state prepare and QST

In [64]:
import numpy as np
import asyncio
import qulab
from qulab.tools.gate_sequence_simulator._GSS import *

class N_qubit_state_prepare_and_QST(qulab.Application):
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0','Qubit1'],
                        'pre_drive': None,
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 获取QST需要的所有预操作
        base = ['I','X','X2p','Y2p']
        N = len(setting_dict['circuit_list'])
        qubit_idx = []
        qst_gate_list = []
        for idx in range(N):
            qubit_idx.append(int(setting_dict['circuit_list'][idx].split('Qubit')[1]))
            qst_gate_list.append([])
        for idx in range(N):
            temp_gate_list = list()
            for base_gate in base:
                for repeat in range(4**(N-idx-1)):
                    temp_gate_list.append(base_gate+str(qubit_idx[idx]))
            for period in range(4**idx):
                qst_gate_list[idx].append(temp_gate_list)
            qst_gate_list[idx] = np.hstack(qst_gate_list[idx])
        qst_gate_list = np.array(qst_gate_list).T
        
        ## 对QST需要的每种预操作情况进行测量
        for qst_gate in qst_gate_list:
            setting_dict['gate_list'] = list(setting_dict['pre_drive'])
            for single_gate in qst_gate:
                setting_dict['gate_list'].append(single_gate)
            ## 设置awg
            drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
            drive_pulse = await drive_pulse_app.done()
            drive_pulse = drive_pulse[0]
            set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
            state = await set_drive_awg_app.done()

            ## 采集卡进行一次数据采集
            ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
            I, Q = await ats_app.done()
            setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
            process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                        ).with_settings(setting_dict['result_process_app_setting'])
            result = await process_app.done()
            
            # 获取理论结果
            simulator = GSS()
            simulator.gate_list = setting_dict['gate_list']
            theory_result = simulator.get_final_probability()
            yield result[1], theory_result
    
    @staticmethod
    def plot(fig, data):
        result, theory = data
        ax = fig.add_subplot(111)
        if isinstance(result,np.ndarray):
            ax.plot(result[-1],'r*',markersize ='10')
            ax.bar(range(theory[-1].size),theory[-1],color='lightskyblue')
        else:
            ax.plot(result,'r*',markersize ='10')
            ax.bar(range(theory.size),theory,color='lightskyblue')
        ax.set_ylabel('amp/probability(a.u)')
        ax.set_xlabel('qubit_idx/engien_state_idx')

In [65]:
N_qubit_state_prepare_and_QST.save(package='N_bits')

# Ramsey Tomography Oscilloscope

## find the optimised delay

In [95]:
import numpy as np
import asyncio
import qulab
from qulab.tools.wavedata import *
from qulab.tools.cliffordgroup import *

class RTO_find_delay(qulab.Application):
    '''
    找RTO测试的最佳delay时间，同时也得到驱动频率偏移量
    app.sweep(delay_time): 待测试的delay序列
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        
        ## 对不同的delay time进行测试，得到py/px变化最敏感的工作点
        qubit_idx = int(setting_dict['circuit_list'][0].split('Qubit')[1])
        drive_freq = setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency']
        async for x in self.sweep['delay_time']:
            ### 分别测量py和px
            ### 不能一个周期内px、py交替，采集卡无法确认先采集的是哪个
            ### 需要手动分别对采集数据的奇偶项进行平均，得到py和px
            ## 根据delay time生成awg波形
            setting_dict['qubit_info'][setting_dict['circuit_list'][0]]['pi_array'][4] = x
            
            out = [0,0]
            for idx in range(2):
                if idx==0:
                    # 测py
                    setting_dict['gate_list'] = ['X2p'+str(qubit_idx),'I'+str(qubit_idx),'X2n'+str(qubit_idx)]
                else:
                    # 测px
                    setting_dict['gate_list'] = ['X2p'+str(qubit_idx),'I'+str(qubit_idx),'Y2p'+str(qubit_idx)]
                
                ## 根据delay time计算驱动频率
                setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = int(drive_freq-0.25/x)
                ## 设置MW
                set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
                await set_mw_app.done()
                
                ## 设置awg
                drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
                drive_pulse = await drive_pulse_app.done()
                drive_pulse = drive_pulse[0]
                set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
                state = await set_drive_awg_app.done()
                await asyncio.sleep(0.02)
                
                ## 采集卡进行一次数据采集
                ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                I, Q = await ats_app.done()
                
                setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                            ).with_settings(setting_dict['result_process_app_setting'])
                result = await process_app.done()
                if isinstance(result[0], np.ndarray):
                    out[idx] = result[0][0]
                else:
                    out[idx] = result[0]
                    
            ## 根据测量的py/px计算qubit频率在delay time内的变化
            out = np.array(out)*2-1
            delta_f01 = np.arctan(out[0]/out[1])/(2*np.pi*x)
            yield x, delta_f01

    async def set_delay_time(self, x):
        await asyncio.sleep(0.02)

    @staticmethod
    def plot(fig, data):
        x, delta_f01 = data
        ax = fig.add_subplot(111)
        ax.plot(x*1e9, delta_f01/1e3, 'r-*')
        ax.set_xlabel('time (ns)')
        ax.set_ylabel('delta_f01 (kHz)')

In [96]:
RTO_find_delay.save(package='RTO')

## RTO line

In [97]:
import numpy as np
import asyncio
import qulab
from qulab.tools.wavedata import *
from qulab.tools.cliffordgroup import *

class RTO_line(qulab.Application):
    '''
    找RTO测试的最佳delay时间，同时也得到驱动频率偏移量
    app.sweep(delay_time): 待测试的delay序列
    '''
    async def work(self):
        ## 取测试相关参数和设备信息
        setting_dict = {'RTO_points': 1000,
                        'delay_time': 50e-9,
                        'circuit_list': ['Qubit0',],
                        'mw_info':None,
                        'qubit_info':None,
                        'two_bit_gate_info': None,
                        'child_app': 'ATS_PACK.ats_acquire',
                        'child_app_setting': dict(),
                        'result_process_app': 'ATS_PACK.ats_result_process',
                        'result_process_app_setting': dict()}
        for setting in self.settings:
            if setting in setting_dict:
                setting_dict[setting] = self.settings.get(setting, 1)
        ## 根据delay time计算驱动频率
        setting_dict['mw_info']['drive_info'][setting_dict['circuit_list'][0]]['frequency'] = int(drive_freq-0.25/setting_dict['delay_time'])
        ## 设置MW
        set_mw_app = qulab.make_app('MW.set_mw',parent=self).with_settings(setting_dict)
        await set_mw_app.done()
        
        ## 
        for x in range(setting_dict['RTO_points']):
            ### 分别测量py和px
            out = [0,0]
            for idx in range(2):
                if idx==0:
                    # 测py
                    setting_dict['gate_list'] = ['X2p'+str(qubit_idx),'I'+str(qubit_idx),'X2n'+str(qubit_idx)]
                else:
                    # 测px
                    setting_dict['gate_list'] = ['X2p'+str(qubit_idx),'I'+str(qubit_idx),'Y2p'+str(qubit_idx)]
                    
                ## 设置awg
                drive_pulse_app = qulab.make_app('base.pulse_sequence_generate',parent=self).with_settings(setting_dict)
                drive_pulse = await drive_pulse_app.done()
                drive_pulse = drive_pulse[0]
                set_drive_awg_app = qulab.make_app('base.map_drive_pulse',parent=self).with_settings({'drive_pulse': drive_pulse})
                state = await set_drive_awg_app.done()
                await asyncio.sleep(0.02)
                
                ## 采集卡进行一次数据采集
                ats_app = qulab.make_app(setting_dict['child_app'],parent=self).with_settings(setting_dict['child_app_setting'])
                I, Q = await ats_app.done()
                setting_dict['result_process_app_setting'].update({'I': I, 'Q': Q})
                process_app = qulab.make_app(setting_dict['result_process_app'],parent=self
                                            ).with_settings(setting_dict['result_process_app_setting'])
                result = await process_app.done()
                if isinstance(result[0], np.ndarray):
                    out[idx] = result[0][0]
                else:
                    out[idx] = result[0]
                    
            ## 根据测量的py/px计算qubit频率在delay time内的变化
            out = np.array(out)*2-1
            delta_f01 = np.arctan(out[0]/out[1])/(2*np.pi*setting_dict['delay_time'])
            yield x, delta_f01

    @staticmethod
    def plot(fig, data):
        x, delta_f01 = data
        ax = fig.add_subplot(111)
        ax.plot(x, delta_f01/1e3, 'r-')
        ax.set_xlabel('points')
        ax.set_ylabel('delta_f01 (kHz)')

In [98]:
RTO_line.save(package='RTO')